In [ ]:
import cv2
import numpy as np

img=cv2.imread("marm.jpg")
center=(200,100)
width=200
height=100
angle=10
rect=(center, (width, height), angle)
box = cv2.boxPoints(rect).astype(np.int0)
print(box)
cv2.drawContours(img,[box],0,(255,0,0),3)
cv2.imshow("img",img)
cv2.waitKey()
cv2.destroyAllWindows()
type(box[2])

In [ ]:
import cv2
import numpy as np
import math  as mt
import serial
import serial.tools.list_ports
import string
import binascii
import time

img=cv2.imread("marm.jpg")
#图像缩一半便于屏幕显示处理
shape=img.shape
print(shape)
H=int(shape[0]/3)
W=int(shape[1]/3)
print(W,H)
img=cv2.resize(img,(W,H))
img= cv2.copyMakeBorder(img,300,0,300,0,cv2.BORDER_CONSTANT,value=(100,100,0))
imgcon=img.copy()
zcenter=(470,300)

#电机模型,
class mmotor():
    def __init__(self,center,angle,startangle,maxangle):
        self.center=center
        self.angle=angle
        self.startangle=startangle
        self.maxangle=maxangle
        self.relaangle=0
        self.facetome=False
        
    def setcenter(self,center):
        self.center=center
        
    def draw(self,angle):
        self.angle=angle
        color=(0,255,0)
        r=30
        cv2.circle(img,self.center,r,color,-1)
        rect=(self.center,(2*r,4),angle)
        box = cv2.boxPoints(rect).astype(np.int0)
        cv2.drawContours(img,[box],0,(255,0,0),3)
        #画z方向
        cv2.circle(img,zcenter,3,(255,0,0),-1)
        
    def drawrelative(self,relaangle):
        if relaangle >self.maxangle:
            relaangle=self.maxangle
        self.relaangle=relaangle
        self.angle=self.startangle+self.relaangle
        self.draw(self.angle)

# 臂模型
class mlever():
    def __init__(self,center,width,high,angle):
        self.center=center
        self.width=width
        self.angle=angle
        self.height=high
        self.jointpoint=(0,0)
        self.rightend=(width,0)
        self.rightendr=10
        self.zcenter=0
        self.zwidth=0
        self.zheight=20
        self.zangle=90
    
    def draw(self):
        global img,imgcon
        height=100
        # 没有连接点，以中心为轴旋转
        if self.jointpoint != (0,0):
            # 有连接点，则重新确定中心，绕连接点转
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            h=int(self.width/2*mt.sin(self.angle/180*mt.pi))
            self.center=(self.jointpoint[0]-w,self.jointpoint[1]-h)
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(self.jointpoint[0]-w*2,self.jointpoint[1]-h*2)
        else:
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(int((box[2][0]+box[3][0])/2),int((box[2][1]+box[3][1])/2))
        cv2.drawContours(img,[box],0,(255,0,0),3)
        #画Z轴的投影
        self.zcenter=self.center
        self.zcenter=(zcenter[0],self.zcenter[0])
        self.zwidth=self.width*mt.cos(self.angle/180*mt.pi)
        self.zangle=90
        zrect=(self.zcenter, (self.zwidth,self.zheight), self.zangle)
        zbox = cv2.boxPoints(zrect).astype(np.int0)
        cv2.drawContours(img,[zbox],0,(0,255,0),3)
        
    def setl(self,center,angle):
        self.center=center
        self.angle=angle
    def setangle(self,angle):
        self.angle=angle    
    def setjointpoint(self,jointpoint):
        self.jointpoint=jointpoint
    def drawrightend(self):
        cv2.circle(img,self.rightend,self.rightendr,(0,0,255),3)

class Marm():
    def __init__(self,m01,m02,m03,ml01,ml02,ml03):
        self.m01=m01
        self.m02=m02
        self.m03=m03        
        self.ml01=ml01
        self.ml02=ml02
        self.ml03=ml03 
        self.angle1=0
        self.angle2=0
        self.angle3=0
        self.ser=None
        self.img=None
        self.imgcon=None
        #将梁2固定在电机2的中心
        self.ml02.setjointpoint(self.m02.center)
        #模拟器初始化
        self.simuinit()
        #串口及机械臂位置初始化
        self.robotinit()
        
    def robotinit(self):
        #打开串口
        ports = list(serial.tools.list_ports.comports())
        print (ports)
        #便利 寻找串口
        for p in ports:
            print (p[1])
            if "serial" in p[1] or "CH340" in p[1]:
                self.ser=serial.Serial(port=p[0])
                if self.ser is not None:
                    print(self.ser)
                    print("serial conected")
                else:
                    print("serial not connected")
                    return
            else:
                print ("No CH340 Device was found connected to the computer")
                return
        if self.ser is None:
            return
        m0=bytes.fromhex('ff 01 00 08 00')# 舵机0速度为 72
        m1=bytes.fromhex('ff 01 01 08 00')# 舵机1速度为 72
        m2=bytes.fromhex('ff 01 02 08 00')# 舵机2速度为 72
        mo0=bytes.fromhex('ff 02 00 dc 05')#设置初始舵机0位置
        mo1=bytes.fromhex('ff 02 01 82 07')#设置初始舵机1的位置
        mo2 = bytes.fromhex('ff 02 02 dc 05')#设置初始舵机2的位置
        mo3 = bytes.fromhex('ff 02 03 dc 05')#设置初始舵机3的位置
        self.ser.write(m0)
        self.ser.write(m1)
        self.ser.write(m2)
        self.ser.write(mo0)
        self.ser.write(mo1)
        self.ser.write(mo2)
        self.ser.write(mo3)
        
    def simuinit(self):
        return

    def xytoq(self,x,y):
        L1=self.ml01.width
        L2=self.ml02.width
        L3=mt.sqrt(y*y+x*x)+0.01
        #L3最长不超过两臂之和
        if L3>L1+L2:
            L3=L1+L2-4
        #L3最短不少于两臂之差
        if L3<abs(L1-L2):
            L3=abs(L1-L2)+4
        p=(L1+L2+L3)/2
        S=mt.sqrt(p*(p-L1)*(p-L2)*(p-L3))
        h=S*2/L3
        Q5=mt.asin(h/L2)/mt.pi*180
        Q4=mt.asin(y/L3)/mt.pi*180
        Q7=mt.asin(h/L1)/mt.pi*180
        Q1=180-Q5-Q7
        Q2=Q5+Q4
        #print("Q1,Q2",Q1,Q2)
        return Q1,Q2
    
    #输入参数为屏幕角度，不考虑电机面对面放置
    def scrdraw(self,angle1,angle2):
        #电机02
        self.m02.drawrelative(angle2)
        #杆02
        self.ml02.setangle(angle2)
        self.ml02.draw()
        #电机01放在杆02的右端
        self.m01.setcenter(self.ml02.rightend)
        self.m01.startangle=180+self.m02.angle
        if angle1<=0:
            angle1=0
        self.m01.drawrelative(angle1)
        #杆01固定在电机01中心，角度与电机01一样
        self.ml01.setjointpoint(self.m01.center)
        self.ml01.setangle(self.m01.angle)
        self.ml01.draw()
        self.ml01.drawrightend()
        #杆03永远保持垂直，中心点位于杆01右端下方
        self.ml03.setangle(90)
        L=int(self.ml03.width/2)
        self.ml03.center=np.add(self.ml01.rightend,(0,L))
        self.ml03.draw()
        self.ml03.drawrightend()
        if self.ser is not None:
            cv2.circle(img,(50,50),10,(0,255,0),3)
        else:
            cv2.circle(img,(50,50),10,(0,0,255),3)
        
    #输入角度考虑电机是否对面装配，然后画在图上   
    def robodraw(self,angle1,angle2):
        mangle1=angle1
        mangle2=angle2
        if self.m01.facetome == True:
            print("facetome")
            mangle1=self.m01.maxangle-angle1
        if self.m02.facetome == True:
            mangle2=slef.m02.maxangle-angle2
        print("mangle1,mangle2",mangle1,mangle2)
        self.scrdraw(mangle1,mangle2)
        
    def drawpwm(self,pwm1,pwm2):
        angle1disp=int((pwm1-500)/2000*180)
        angle2disp=int((pwm2-500)/2000*180)
        angel1=int(angle1disp*self.m01.maxangle/180)
        angel2=int(angle2disp*self.m02.maxangle/180)
        
        A1=180+50-(angel1+angel2)
        print("angle1disp,angle2disp,A1,%x,%x,"%(pwm1,pwm2),angle1disp,angle2disp,A1)
        self.robodraw(A1,angel2)
    def xytopwm(self,x,y):
        q1,q2=self.xytoq(x,y)
        # q1=180+50-(q2+pwmq1)
        pwmq1=180+50-(q1+q2)
        pwmq2=q2
        pwm1=int(pwmq1/180*2000+500)
        pwm2=int(pwmq2/180*2000+500)
        return pwm1,pwm2
    
    def serwrite(self,pwm1,pwm2):
        if self.ser is None:
            return
        print(pwm1,pwm2)
        pw1str="%04x"%pwm1
        pw2str="%04x"%pwm2
        print(pw1str,pw2str)
        mo1hexstr="ff 02 01"+" "+pw1str[2:4]+" "+pw1str[0:2]
        mo2hexstr="ff 02 02"+" "+pw2str[2:4]+" "+pw2str[0:2]
        self.ser.write(bytes.fromhex(mo1hexstr))
        self.ser.write(bytes.fromhex(mo2hexstr))
        return
    
    def robotTwin(self,x,y):
        pwm1,pwm2=self.xytopwm(x,y)
        self.drawpwm(pwm1,pwm2)
        self.serwrite(pwm1,pwm2)
    
    def destroy(self):
        if self.ser is not None:
            self.ser.close()
        cv2.destroyAllWindows()
        
#配置零件舵机
mM02=mmotor((170+300,240+300),0,0,180)
mM01=mmotor((0,0),0,0,180)
mM03=mmotor((0,0),0,0,180)
#配置零件梁
mMl01=mlever((0,0),200,10,0)
mMl02=mlever((0,0),200,10,0)
mMl03=mlever((0,0),60,10,0)
#组装成机械臂
marm=Marm(mM01,mM02,mM03,mMl01,mMl02,mMl03)

for i in range(60):
    x=i*10
    y=0
    marm.robotTwin(x,y)
    cv2.imshow("marm",img)
    if cv2.waitKey(100)&0xff == 27:
        break
    #恢复背景
    img=imgcon.copy()

for i in range(60):
    x=600-i*10
    y=0
    marm.robotTwin(x,y)
    cv2.imshow("marm",img)
    if cv2.waitKey(100)&0xff == 27:
        break
    #恢复背景
    img=imgcon.copy()
    
time.sleep(3)
marm.destroy()

In [ ]:
import cv2
import numpy as np
import math as m
img=cv2.imread("marm.jpg")
leftbuttom=(92,131)
width=200
height=100
angle=10
print(m.sin(angle))
print(m.cos(angle))

x=int(m.sin(angle/180*m.pi)*width/2)
y=int(m.cos(angle/188*m.pi)*width/2)
center=leftbuttom+np.array([y,x])
rect=(center, (width, height), angle)
box = cv2.boxPoints(rect).astype(np.int0)
print(box)
cv2.drawContours(img,[box],0,(255,0,0),3)
cv2.imshow("img",img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import math  as mt
import serial
import serial.tools.list_ports
import string
import binascii
import time

img=cv2.imread("marm.jpg")
img9sea=cv2.imread("9seacut.png")
img9sea=cv2.resize(img9sea,(500,350))
#图像缩一半便于屏幕显示处理
shape=img.shape
print(shape)
H=int(shape[0]/3)
W=int(shape[1]/3)
print(W,H)

shape2=img9sea.shape
print(shape2)
H2=shape2[0]
W2=shape2[1]

img=cv2.resize(img,(W,H))
img= cv2.copyMakeBorder(img,300,0,300,0,cv2.BORDER_CONSTANT,value=(100,100,0))
img[200:200+H2,200:200+W2]=img9sea
imgcon=img.copy()

#顶视图平面中点
zcenter=(470,300)

#电机模型,
class mmotor():
    def __init__(self,center,angle,startangle,minangle,maxangle):
        self.center=center
        self.angle=angle
        #电机中心点随杠杆旋转时电机0位与水平线的夹角
        self.startangle=startangle
        #电机中点为90度，顺时针方向为正，最小点和最大点角度
        self.maxangle=maxangle
        self.minangle=minangle
        self.relaangle=0
        self.facetome=False
        self.zcenter=(0,0)
        
    def setcenter(self,center):
        self.center=center
        
    def draw(self,angle):
        self.angle=angle
        #通过舵机角度270判断是否为底座舵机，画图时大一些
        if self.maxangle-self.startangle >200:
            r=50
            color=(0,255,0)
            cv2.circle(img,self.center,r,color,3)
            rect=(self.center,(2*r,4),angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            cv2.drawContours(img,[box],0,color,3)
        else:
            r=20
            color=(155,0,0)
            cv2.circle(img,self.center,r,color,3)
            rect=(self.center,(2*r,4),angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            cv2.drawContours(img,[box],0,color,3)
      
    def drawrelative(self,relaangle):
        if relaangle >self.maxangle:
            relaangle=self.maxangle
        self.relaangle=relaangle
        self.angle=self.startangle+self.relaangle
        self.draw(self.angle)

# 臂模型
class mlever():
    def __init__(self,center,width,high,angle):
        self.center=center
        self.width=width
        self.angle=angle
        self.height=high
        self.jointpoint=(0,0)
        self.zjointpoint=(0,0)
        self.rightend=(width,0)
        self.zrightend=(0,0)
        self.rightendr=10
        self.zcenter=(0,0)
        self.zwidth=100
        self.zheight=10
        self.zangle=60
    
    def draw(self):
        global img,imgcon
        height=100
        # 没有连接点，以中心为轴旋转
        if self.jointpoint != (0,0):
            # 有连接点，则重新确定中心，绕连接点转 w,h为中心点相对坐标
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            h=int(self.width/2*mt.sin(self.angle/180*mt.pi))
            #z轴的投影长度为中心点坐标的一倍
            self.zwidth=2*w
            self.center=(self.jointpoint[0]-w,self.jointpoint[1]-h)
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(self.jointpoint[0]-w*2,self.jointpoint[1]-h*2)
        else:
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            #z轴的投影长度为中心点坐标的一倍
            self.zwidth=2*w
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(int((box[2][0]+box[3][0])/2),int((box[2][1]+box[3][1])/2))
        cv2.drawContours(img,[box],0,(155,0,0),3)
        #画Z轴的投影
        if self.zjointpoint != (0,0):
            # 有连接点，则重新确定中心，绕连接点转
            print("ZJ point",self.zjointpoint)
            w=int(self.zwidth/2*mt.cos(self.zangle/180*mt.pi))
            h=int(self.zwidth/2*mt.sin(self.zangle/180*mt.pi))
            self.zcenter=(self.zjointpoint[0]-w,self.zjointpoint[1]-h)
        else:
            self.zcenter=zcenter
            pass
        zrect=(self.zcenter, (self.zwidth,self.zheight), self.zangle)
        print("zrect",zrect)
        zbox = cv2.boxPoints(zrect).astype(np.int0)
        self.zrightend=(int((zbox[0][0]+zbox[1][0])/2),int((zbox[0][1]+zbox[1][1])/2))
        cv2.drawContours(img,[zbox],0,(0,255,0),3)
        
    def setl(self,center,angle):
        self.center=center
        self.angle=angle
    def setangle(self,angle):
        self.angle=angle    
    def setjointpoint(self,jointpoint):
        self.jointpoint=jointpoint
    def setzjointpoint(self,jointpoint):
        self.zjointpoint=jointpoint
    def setzangle(self,angle):
        self.zangle=angle
    def drawrightend(self):
        cv2.circle(img,self.rightend,self.rightendr,(0,0,255),3)
        cv2.circle(img,self.zrightend,self.rightendr,(0,255,0),3)

class Marm():
    def __init__(self,m00,m01,m02,m03,ml01,ml02,ml03):
        self.m00=m00
        self.m01=m01
        self.m02=m02
        self.m03=m03        
        self.ml01=ml01
        self.ml02=ml02
        self.ml03=ml03 
        self.ser=None
        self.img=None
        self.imgcon=None
        

        #模拟器初始化
        self.simuinit()
        #串口及机械臂位置初始化
        self.robotinit()
        
    def robotinit(self):
        #打开串口
        ports = list(serial.tools.list_ports.comports())
        print (ports)
        #便利 寻找串口
        for p in ports:
            print (p[1])
            if "serial" in p[1] or "CH340" in p[1]:
                self.ser=serial.Serial(port=p[0])
                if self.ser is not None:
                    print(self.ser)
                    print("serial conected")
                else:
                    print("serial not connected")
                    return
            else:
                print ("No CH340 Device was found connected to the computer")
                return
        if self.ser is None:
            return
        m0=bytes.fromhex('ff 01 00 08 00')# 舵机0速度为 72
        m1=bytes.fromhex('ff 01 01 08 00')# 舵机1速度为 72
        m2=bytes.fromhex('ff 01 02 08 00')# 舵机2速度为 72
        mo0=bytes.fromhex('ff 02 00 dc 05')#设置初始舵机0位置
        mo1=bytes.fromhex('ff 02 01 82 07')#设置初始舵机1的位置
        mo2 = bytes.fromhex('ff 02 02 dc 05')#设置初始舵机2的位置
        mo3 = bytes.fromhex('ff 02 03 dc 05')#设置初始舵机3的位置
        self.ser.write(m0)
        self.ser.write(m1)
        self.ser.write(m2)
        self.ser.write(mo0)
        self.ser.write(mo1)
        self.ser.write(mo2)
        self.ser.write(mo3)
        
    def simuinit(self):
        return
    
    def xztoq0(self,x,z):
        r=mt.sqrt(x*x+z*z+0.01)
        if z >=0:
            q0=180-mt.acos(z/r)/mt.pi*180
        else:
            q0=mt.acos(abs(z)/r)/mt.pi*180
        return q0
    
    def xytoq12(self,x,y):
        L1=self.ml01.width
        L2=self.ml02.width
        L3=mt.sqrt(y*y+x*x)+0.01
        #L3最长不超过两臂之和
        if L3>L1+L2:
            L3=L1+L2-4
        #L3最短不少于两臂之差
        if L3<abs(L1-L2):
            L3=abs(L1-L2)+4
        p=(L1+L2+L3)/2
        S=mt.sqrt(p*(p-L1)*(p-L2)*(p-L3))
        h=S*2/L3
        Q5=mt.asin(h/L2)/mt.pi*180
        Q4=mt.asin(y/L3)/mt.pi*180
        Q7=mt.asin(h/L1)/mt.pi*180
        Q1=180-Q5-Q7
        Q2=Q5+Q4
        #print("Q1,Q2",Q1,Q2)
        return Q1,Q2
    
    #输入参数为屏幕角度，不考虑电机面对面放置,考虑杆的移动相对位置
    def scrdraw(self,angle0,angle1,angle2):
        #电机0在底盘
        self.m00.drawrelative(angle0)
        #杆02与电机0中心在z轴方向相连
        self.ml02.setzjointpoint(self.m00.center)
        self.ml02.setzangle(angle0)
        #电机02
        self.m02.drawrelative(angle2)
         #杆02,与m02的圆心相连
        self.ml02.setangle(angle2)
        self.ml02.draw()
        self.ml02.setjointpoint(self.m02.center)
        #电机01放在杆02的右端
        self.m01.setcenter(self.ml02.rightend)
        self.m01.zcenter=self.ml02.zrightend
        print("m01 zcenter is",self.m01.zcenter)
        self.m01.startangle=180+self.m02.angle
        if angle1<=0:
            angle1=0
        self.m01.drawrelative(angle1)
        #杆01固定在电机01中心，角度与电机01一样
        self.ml01.setjointpoint(self.m01.center)
        self.ml01.setangle(self.m01.angle)
        self.ml01.setzjointpoint(self.m01.zcenter)
        self.ml01.setzangle(angle0)
        self.ml01.draw()
        self.ml01.drawrightend()
        #杆03永远保持垂直，中心点位于杆01右端下方
        self.ml03.setangle(90)
        L=int(self.ml03.width/2)
        self.ml03.center=np.add(self.ml01.rightend,(0,L))
        self.ml03.setzjointpoint(self.ml01.zrightend)
        self.ml03.setzangle(angle0)
        self.ml03.draw()
        self.ml03.drawrightend()
        #显示串口连接状态红未连接绿连接
        if self.ser is not None:
            cv2.circle(img,(50,50),10,(0,255,0),3)
        else:
            cv2.circle(img,(50,50),10,(0,0,255),3)
        
    #输入角度考虑电机是否对面装配，从而是逆时针旋转，然后画在图上   
    def robodraw(self,angle0,angle1,angle2):
        mangle0=angle0
        mangle1=angle1
        mangle2=angle2
        if self.m00.facetome == True:
            print("facetome")
            mangle0=self.m00.maxangle-angle0
        if self.m01.facetome == True:
            print("facetome")
            mangle1=self.m01.maxangle-angle1
        if self.m02.facetome == True:
            print("facetome")
            mangle2=slef.m02.maxangle-angle2
        print("mangel0,mangle1,mangle2",mangle0,mangle1,mangle2)
        self.scrdraw(mangle0,mangle1,mangle2)
    
    #输入PWM值画机械臂。对于非180度舵机实施缩放处理。对零位非0度的情况进行处理
    def drawpwm(self,pwm0,pwm1,pwm2):
        angle0pc=int((pwm0-500)/2000*180)
        angle0pc=180-angle0pc
        angle1pc=int((pwm1-500)/2000*180)
        angle2pc=int((pwm2-500)/2000*180)
        angel0=int(angle0pc*(self.m00.maxangle-self.m00.minangle)/180)+self.m00.minangle
        angel1=int(angle1pc*(self.m01.maxangle-self.m01.minangle)/180)+self.m01.minangle
        angel2=int(angle2pc*(self.m02.maxangle-self.m02.minangle)/180)+self.m02.minangle
        A1=180+50-(angel1+angel2)
        print("angel0,A1,angel2",angel0,A1,angel2)
        self.robodraw(angel0,A1,angel2)
        
    def xytopwm(self,x,y,z):
        r=mt.sqrt(x*x+z*z)
        q1,q2=self.xytoq12(r,y)
        # q1=180+50-(q2+pwmq1)
        pwmq1=180+50-(q1+q2)
        pwmq2=q2
        pwm1=int(pwmq1/180*2000+500)
        pwm2=int(pwmq2/180*2000+500)
        
        q0=self.xztoq0(x,z)
        qq0=q0-self.m00.minangle
        pwmq0=qq0/(self.m00.maxangle-self.m00.minangle)*180
        pwm0=int(pwmq0/180*2000+500)
        print("pwm0,pwm1,pwm2",pwm0,pwm1,pwm2)
        return pwm0,pwm1,pwm2
    
    def serwrite(self,pwm0,pwm1,pwm2):
        if self.ser is None:
            return
        print(pwm0,pwm1,pwm2)
        pw0str="%04x"%pwm0
        pw1str="%04x"%pwm1
        pw2str="%04x"%pwm2
        print(pw0str,pw1str,pw2str)
        mo0hexstr="ff 02 00"+" "+pw0str[2:4]+" "+pw0str[0:2]
        mo1hexstr="ff 02 01"+" "+pw1str[2:4]+" "+pw1str[0:2]
        mo2hexstr="ff 02 02"+" "+pw2str[2:4]+" "+pw2str[0:2]
        self.ser.write(bytes.fromhex(mo0hexstr))
        self.ser.write(bytes.fromhex(mo1hexstr))
        self.ser.write(bytes.fromhex(mo2hexstr))
        return
    
    def robotTwin(self,x,y,z):
        pwm0,pwm1,pwm2=self.xytopwm(x,y,-1*z)
        self.drawpwm(pwm0,pwm1,pwm2)
        self.serwrite(pwm0,pwm1,pwm2)
    def show(self):
        global img
        cv2.imshow("marm",img)
        cv2.waitKey(300)
        #恢复背景
        img=imgcon.copy()
    def home(self):
        self.robotTwin(0,0,0)
        self.show()
        cv2.waitKey(5000)
    def destroy(self):
        if self.ser is not None:
            self.ser.close()
        cv2.destroyAllWindows()
        
#配置零件舵机
mM00=mmotor((170+300,240+300),0,0,-45,225)
mM02=mmotor((170+300,240+300),0,0,0,180)
mM01=mmotor((0,0),0,0,0,180)
mM03=mmotor((0,0),0,0,0,180)
#配置零件梁
mMl01=mlever((0,0),200,10,0)
mMl02=mlever((0,0),200,10,0)
mMl03=mlever((0,0),60,10,0)
#组装成机械臂
marm=Marm(mM00,mM01,mM02,mM03,mMl01,mMl02,mMl03)

marm.home()
for i in range(-25,25):
    marm.robotTwin(300,0,i*10)
    marm.show()    
    time.sleep(1)
for i in range(-25,25):
    marm.robotTwin(100,0,i*10)
    marm.show()    
    time.sleep(1)
    
marm.destroy()

In [ ]:
import cv2
import numpy as np
import math  as mt
import serial
import serial.tools.list_ports
import string
import binascii
import time

img=cv2.imread("marm.jpg")
img9sea=cv2.imread("9seacut.png")
img9sea=cv2.resize(img9sea,(500,350))
#图像缩一半便于屏幕显示处理
shape=img.shape
print(shape)
H=int(shape[0]/3)
W=int(shape[1]/3)
print(W,H)

shape2=img9sea.shape
print(shape2)
H2=shape2[0]
W2=shape2[1]

img=cv2.resize(img,(W,H))
img= cv2.copyMakeBorder(img,300,0,300,0,cv2.BORDER_CONSTANT,value=(100,100,0))
img[200:200+H2,200:200+W2]=img9sea
imgcon=img.copy()

#顶视图平面中点
zcenter=(470,300)

#电机模型,
class mmotor():
    def __init__(self,center,angle,startangle,minangle,maxangle):
        self.center=center
        self.angle=angle
        #电机中心点随杠杆旋转时电机0位与水平线的夹角
        self.startangle=startangle
        #电机中点为90度，顺时针方向为正，最小点和最大点角度
        self.maxangle=maxangle
        self.minangle=minangle
        self.relaangle=0
        self.facetome=False
        self.zcenter=(0,0)
        
    def setcenter(self,center):
        self.center=center
        
    def draw(self,angle):
        self.angle=angle
        #通过舵机角度270判断是否为底座舵机，画图时大一些
        if self.maxangle-self.startangle >200:
            r=50
            color=(0,255,0)
            cv2.circle(img,self.center,r,color,3)
            rect=(self.center,(2*r,4),angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            cv2.drawContours(img,[box],0,color,3)
        else:
            r=20
            color=(155,0,0)
            cv2.circle(img,self.center,r,color,3)
            rect=(self.center,(2*r,4),angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            cv2.drawContours(img,[box],0,color,3)
      
    def drawrelative(self,relaangle):
        if relaangle >self.maxangle:
            relaangle=self.maxangle
        self.relaangle=relaangle
        self.angle=self.startangle+self.relaangle
        self.draw(self.angle)

# 臂模型
class mlever():
    def __init__(self,center,width,high,angle):
        self.center=center
        self.width=width
        self.angle=angle
        self.height=high
        self.jointpoint=(0,0)
        self.zjointpoint=(0,0)
        self.rightend=(width,0)
        self.zrightend=(0,0)
        self.rightendr=10
        self.zcenter=(0,0)
        self.zwidth=100
        self.zheight=10
        self.zangle=60
    
    def draw(self):
        global img,imgcon
        height=100
        # 没有连接点，以中心为轴旋转
        if self.jointpoint != (0,0):
            # 有连接点，则重新确定中心，绕连接点转 w,h为中心点相对坐标
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            h=int(self.width/2*mt.sin(self.angle/180*mt.pi))
            #z轴的投影长度为中心点坐标的一倍
            self.zwidth=2*w
            self.center=(self.jointpoint[0]-w,self.jointpoint[1]-h)
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(self.jointpoint[0]-w*2,self.jointpoint[1]-h*2)
        else:
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            #z轴的投影长度为中心点坐标的一倍
            self.zwidth=2*w
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(int((box[2][0]+box[3][0])/2),int((box[2][1]+box[3][1])/2))
        cv2.drawContours(img,[box],0,(155,0,0),3)
        #画Z轴的投影
        if self.zjointpoint != (0,0):
            # 有连接点，则重新确定中心，绕连接点转
            print("ZJ point",self.zjointpoint)
            w=int(self.zwidth/2*mt.cos(self.zangle/180*mt.pi))
            h=int(self.zwidth/2*mt.sin(self.zangle/180*mt.pi))
            self.zcenter=(self.zjointpoint[0]-w,self.zjointpoint[1]-h)
        else:
            self.zcenter=zcenter
            pass
        zrect=(self.zcenter, (self.zwidth,self.zheight), self.zangle)
        print("zrect",zrect)
        zbox = cv2.boxPoints(zrect).astype(np.int0)
        self.zrightend=(int((zbox[0][0]+zbox[1][0])/2),int((zbox[0][1]+zbox[1][1])/2))
        cv2.drawContours(img,[zbox],0,(0,255,0),3)
        
    def setl(self,center,angle):
        self.center=center
        self.angle=angle
    def setangle(self,angle):
        self.angle=angle    
    def setjointpoint(self,jointpoint):
        self.jointpoint=jointpoint
    def setzjointpoint(self,jointpoint):
        self.zjointpoint=jointpoint
    def setzangle(self,angle):
        self.zangle=angle
    def drawrightend(self):
        cv2.circle(img,self.rightend,self.rightendr,(0,0,255),3)
        cv2.circle(img,self.zrightend,self.rightendr,(0,255,0),3)

class Marm():
    def __init__(self,m00,m01,m02,m03,ml01,ml02,ml03):
        self.m00=m00
        self.m01=m01
        self.m02=m02
        self.m03=m03        
        self.ml01=ml01
        self.ml02=ml02
        self.ml03=ml03 
        self.ser=None
        self.img=None
        self.imgcon=None
        

        #模拟器初始化
        self.simuinit()
        #串口及机械臂位置初始化
        self.robotinit()
        
    def robotinit(self):
        #打开串口
        ports = list(serial.tools.list_ports.comports())
        print (ports)
        #便利 寻找串口
        for p in ports:
            print (p[1])
            if "serial" in p[1] or "CH340" in p[1]:
                self.ser=serial.Serial(port=p[0])
                if self.ser is not None:
                    print(self.ser)
                    print("serial conected")
                else:
                    print("serial not connected")
                    return
            else:
                print ("No CH340 Device was found connected to the computer")
                return
        if self.ser is None:
            return
        m0=bytes.fromhex('ff 01 00 08 00')# 舵机0速度为 72
        m1=bytes.fromhex('ff 01 01 08 00')# 舵机1速度为 72
        m2=bytes.fromhex('ff 01 02 08 00')# 舵机2速度为 72
        mo0=bytes.fromhex('ff 02 00 dc 05')#设置初始舵机0位置
        mo1=bytes.fromhex('ff 02 01 82 07')#设置初始舵机1的位置
        mo2 = bytes.fromhex('ff 02 02 dc 05')#设置初始舵机2的位置
        mo3 = bytes.fromhex('ff 02 03 dc 05')#设置初始舵机3的位置
        self.ser.write(m0)
        self.ser.write(m1)
        self.ser.write(m2)
        self.ser.write(mo0)
        self.ser.write(mo1)
        self.ser.write(mo2)
        self.ser.write(mo3)
        
    def simuinit(self):
        return
    
    def xztoq0(self,x,z):
        r=mt.sqrt(x*x+z*z+0.01)
        if z >=0:
            q0=180-mt.acos(z/r)/mt.pi*180
        else:
            q0=mt.acos(abs(z)/r)/mt.pi*180
        return q0
    
    def xytoq12(self,x,y):
        L1=self.ml01.width
        L2=self.ml02.width
        L3=mt.sqrt(y*y+x*x)+0.01
        #L3最长不超过两臂之和
        if L3>L1+L2:
            L3=L1+L2-4
        #L3最短不少于两臂之差
        if L3<abs(L1-L2):
            L3=abs(L1-L2)+4
        p=(L1+L2+L3)/2
        S=mt.sqrt(p*(p-L1)*(p-L2)*(p-L3))
        h=S*2/L3
        Q5=mt.asin(h/L2)/mt.pi*180
        Q4=mt.asin(y/L3)/mt.pi*180
        Q7=mt.asin(h/L1)/mt.pi*180
        Q1=180-Q5-Q7
        Q2=Q5+Q4
        #print("Q1,Q2",Q1,Q2)
        return Q1,Q2
    
    #输入参数为屏幕角度，不考虑电机面对面放置,考虑杆的移动相对位置
    def scrdraw(self,angle0,angle1,angle2):
        #电机0在底盘
        self.m00.drawrelative(angle0)
        #杆02与电机0中心在z轴方向相连
        self.ml02.setzjointpoint(self.m00.center)
        self.ml02.setzangle(angle0)
        #电机02
        self.m02.drawrelative(angle2)
         #杆02,与m02的圆心相连
        self.ml02.setangle(angle2)
        self.ml02.draw()
        self.ml02.setjointpoint(self.m02.center)
        #电机01放在杆02的右端
        self.m01.setcenter(self.ml02.rightend)
        self.m01.zcenter=self.ml02.zrightend
        print("m01 zcenter is",self.m01.zcenter)
        self.m01.startangle=180+self.m02.angle
        if angle1<=0:
            angle1=0
        self.m01.drawrelative(angle1)
        #杆01固定在电机01中心，角度与电机01一样
        self.ml01.setjointpoint(self.m01.center)
        self.ml01.setangle(self.m01.angle)
        self.ml01.setzjointpoint(self.m01.zcenter)
        self.ml01.setzangle(angle0)
        self.ml01.draw()
        self.ml01.drawrightend()
        #杆03永远保持垂直，中心点位于杆01右端下方
        self.ml03.setangle(90)
        L=int(self.ml03.width/2)
        self.ml03.center=np.add(self.ml01.rightend,(0,L))
        self.ml03.setzjointpoint(self.ml01.zrightend)
        self.ml03.setzangle(angle0)
        self.ml03.draw()
        self.ml03.drawrightend()
        #显示串口连接状态红未连接绿连接
        if self.ser is not None:
            cv2.circle(img,(50,50),10,(0,255,0),3)
        else:
            cv2.circle(img,(50,50),10,(0,0,255),3)
        
    #输入角度考虑电机是否对面装配，从而是逆时针旋转，然后画在图上   
    def robodraw(self,angle0,angle1,angle2):
        mangle0=angle0
        mangle1=angle1
        mangle2=angle2
        if self.m00.facetome == True:
            print("facetome")
            mangle0=self.m00.maxangle-angle0
        if self.m01.facetome == True:
            print("facetome")
            mangle1=self.m01.maxangle-angle1
        if self.m02.facetome == True:
            print("facetome")
            mangle2=slef.m02.maxangle-angle2
        print("mangel0,mangle1,mangle2",mangle0,mangle1,mangle2)
        self.scrdraw(mangle0,mangle1,mangle2)
    
    #输入PWM值画机械臂。对于非180度舵机实施缩放处理。对零位非0度的情况进行处理
    def drawpwm(self,pwm0,pwm1,pwm2):
        angle0pc=int((pwm0-500)/2000*180)
        angle0pc=180-angle0pc
        angle1pc=int((pwm1-500)/2000*180)
        angle2pc=int((pwm2-500)/2000*180)
        angel0=int(angle0pc*(self.m00.maxangle-self.m00.minangle)/180)+self.m00.minangle
        angel1=int(angle1pc*(self.m01.maxangle-self.m01.minangle)/180)+self.m01.minangle
        angel2=int(angle2pc*(self.m02.maxangle-self.m02.minangle)/180)+self.m02.minangle
        A1=180+50-(angel1+angel2)
        print("angel0,A1,angel2",angel0,A1,angel2)
        self.robodraw(angel0,A1,angel2)
        
    def xytopwm(self,x,y,z):
        r=mt.sqrt(x*x+z*z)
        q1,q2=self.xytoq12(r,y)
        # q1=180+50-(q2+pwmq1)
        pwmq1=180+50-(q1+q2)
        pwmq2=q2
        pwm1=int(pwmq1/180*2000+500)
        pwm2=int(pwmq2/180*2000+500)
        
        q0=self.xztoq0(x,z)
        qq0=q0-self.m00.minangle
        pwmq0=qq0/(self.m00.maxangle-self.m00.minangle)*180
        pwm0=int(pwmq0/180*2000+500)
        print("pwm0,pwm1,pwm2",pwm0,pwm1,pwm2)
        return pwm0,pwm1,pwm2
    
    def serwrite(self,pwm0,pwm1,pwm2):
        if self.ser is None:
            return
        print(pwm0,pwm1,pwm2)
        pw0str="%04x"%pwm0
        pw1str="%04x"%pwm1
        pw2str="%04x"%pwm2
        print(pw0str,pw1str,pw2str)
        mo0hexstr="ff 02 00"+" "+pw0str[2:4]+" "+pw0str[0:2]
        mo1hexstr="ff 02 01"+" "+pw1str[2:4]+" "+pw1str[0:2]
        mo2hexstr="ff 02 02"+" "+pw2str[2:4]+" "+pw2str[0:2]
        self.ser.write(bytes.fromhex(mo0hexstr))
        self.ser.write(bytes.fromhex(mo1hexstr))
        self.ser.write(bytes.fromhex(mo2hexstr))
        return
    
    def robotTwin(self,x,y,z):
        pwm0,pwm1,pwm2=self.xytopwm(x,y,-1*z)
        self.drawpwm(pwm0,pwm1,pwm2)
        self.serwrite(pwm0,pwm1,pwm2)
    def show(self):
        global img
        cv2.imshow("marm",img)
        cv2.waitKey(300)
        #恢复背景
        img=imgcon.copy()
    def home(self):
        self.robotTwin(0,0,0)
        self.show()
        cv2.waitKey(5000)
    def destroy(self):
        if self.ser is not None:
            self.ser.close()
        cv2.destroyAllWindows()
        
#配置零件舵机
mM00=mmotor((170+300,240+300),0,0,-45,225)
mM02=mmotor((170+300,240+300),0,0,0,180)
mM01=mmotor((0,0),0,0,0,180)
mM03=mmotor((0,0),0,0,0,180)
#配置零件梁
mMl01=mlever((0,0),200,10,0)
mMl02=mlever((0,0),200,10,0)
mMl03=mlever((0,0),60,10,0)
#组装成机械臂
marm=Marm(mM00,mM01,mM02,mM03,mMl01,mMl02,mMl03)

marm.home()
for i in range(-25,25):
    marm.robotTwin(300,0,i*10)
    marm.show()    
    time.sleep(1)
for i in range(-25,25):
    marm.robotTwin(100,0,i*10)
    marm.show()    
    time.sleep(1)
    
marm.destroy()

# x轴方向移动时贴地面有高度差，找原因并调整

In [ ]:
import cv2
import numpy as np
import math  as mt
import serial
import serial.tools.list_ports
import string
import binascii
import time

img=cv2.imread("marm.jpg")
img9sea=cv2.imread("9seacut.png")
img9sea=cv2.resize(img9sea,(500,350))
#图像缩一半便于屏幕显示处理
shape=img.shape
print(shape)
H=int(shape[0]/3)
W=int(shape[1]/3)
print(W,H)

shape2=img9sea.shape
print(shape2)
H2=shape2[0]
W2=shape2[1]

img=cv2.resize(img,(W,H))
img= cv2.copyMakeBorder(img,300,0,300,0,cv2.BORDER_CONSTANT,value=(100,100,0))
img[200:200+H2,200:200+W2]=img9sea
imgcon=img.copy()

#顶视图平面中点
zcenter=(470,300)

#电机模型,
class mmotor():
    def __init__(self,center,angle,startangle,minangle,maxangle):
        self.center=center
        self.angle=angle
        #电机中心点随杠杆旋转时电机0位与水平线的夹角
        self.startangle=startangle
        #电机中点为90度，顺时针方向为正，最小点和最大点角度
        self.maxangle=maxangle
        self.minangle=minangle
        self.relaangle=0
        self.facetome=False
        self.zcenter=(0,0)
        
    def setcenter(self,center):
        self.center=center
        
    def draw(self,angle):
        self.angle=angle
        #通过舵机角度270判断是否为底座舵机，画图时大一些
        if self.maxangle-self.startangle >200:
            r=50
            color=(0,255,0)
            cv2.circle(img,self.center,r,color,3)
            rect=(self.center,(2*r,4),angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            cv2.drawContours(img,[box],0,color,3)
        else:
            r=20
            color=(155,0,0)
            cv2.circle(img,self.center,r,color,3)
            rect=(self.center,(2*r,4),angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            cv2.drawContours(img,[box],0,color,3)
      
    def drawrelative(self,relaangle):
        if relaangle >self.maxangle:
            relaangle=self.maxangle
        self.relaangle=relaangle
        self.angle=self.startangle+self.relaangle
        self.draw(self.angle)

# 臂模型
class mlever():
    def __init__(self,center,width,high,angle):
        self.center=center
        self.width=width
        self.angle=angle
        self.height=high
        self.jointpoint=(0,0)
        self.zjointpoint=(0,0)
        self.rightend=(width,0)
        self.zrightend=(0,0)
        self.rightendr=10
        self.zcenter=(0,0)
        self.zwidth=100
        self.zheight=10
        self.zangle=60
    
    def draw(self):
        global img,imgcon
        height=100
        # 没有连接点，以中心为轴旋转
        if self.jointpoint != (0,0):
            # 有连接点，则重新确定中心，绕连接点转 w,h为中心点相对坐标
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            h=int(self.width/2*mt.sin(self.angle/180*mt.pi))
            #z轴的投影长度为中心点坐标的一倍
            self.zwidth=2*w
            self.center=(self.jointpoint[0]-w,self.jointpoint[1]-h)
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(self.jointpoint[0]-w*2,self.jointpoint[1]-h*2)
        else:
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            #z轴的投影长度为中心点坐标的一倍
            self.zwidth=2*w
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(int((box[2][0]+box[3][0])/2),int((box[2][1]+box[3][1])/2))
        cv2.drawContours(img,[box],0,(155,0,0),3)
        #画Z轴的投影
        if self.zjointpoint != (0,0):
            # 有连接点，则重新确定中心，绕连接点转
            print("ZJ point",self.zjointpoint)
            w=int(self.zwidth/2*mt.cos(self.zangle/180*mt.pi))
            h=int(self.zwidth/2*mt.sin(self.zangle/180*mt.pi))
            self.zcenter=(self.zjointpoint[0]-w,self.zjointpoint[1]-h)
        else:
            self.zcenter=zcenter
            pass
        zrect=(self.zcenter, (self.zwidth,self.zheight), self.zangle)
        print("zrect",zrect)
        zbox = cv2.boxPoints(zrect).astype(np.int0)
        self.zrightend=(int((zbox[0][0]+zbox[1][0])/2),int((zbox[0][1]+zbox[1][1])/2))
        cv2.drawContours(img,[zbox],0,(0,255,0),3)
        
    def setl(self,center,angle):
        self.center=center
        self.angle=angle
    def setangle(self,angle):
        self.angle=angle    
    def setjointpoint(self,jointpoint):
        self.jointpoint=jointpoint
    def setzjointpoint(self,jointpoint):
        self.zjointpoint=jointpoint
    def setzangle(self,angle):
        self.zangle=angle
    def drawrightend(self):
        cv2.circle(img,self.rightend,self.rightendr,(0,0,255),3)
        cv2.circle(img,self.zrightend,self.rightendr,(0,255,0),3)

class Marm():
    def __init__(self,m00,m01,m02,m03,ml01,ml02,ml03):
        self.m00=m00
        self.m01=m01
        self.m02=m02
        self.m03=m03        
        self.ml01=ml01
        self.ml02=ml02
        self.ml03=ml03 
        self.ser=None
        self.img=None
        self.imgcon=None
        

        #模拟器初始化
        self.simuinit()
        #串口及机械臂位置初始化
        self.robotinit()
        
    def robotinit(self):
        #打开串口
        ports = list(serial.tools.list_ports.comports())
        print (ports)
        #便利 寻找串口
        for p in ports:
            print (p[1])
            if "serial" in p[1] or "CH340" in p[1]:
                self.ser=serial.Serial(port=p[0])
                if self.ser is not None:
                    print(self.ser)
                    print("serial conected")
                else:
                    print("serial not connected")
                    return
            else:
                print ("No CH340 Device was found connected to the computer")
                return
        if self.ser is None:
            return
        m0=bytes.fromhex('ff 01 00 08 00')# 舵机0速度为 72
        m1=bytes.fromhex('ff 01 01 08 00')# 舵机1速度为 72
        m2=bytes.fromhex('ff 01 02 08 00')# 舵机2速度为 72
        mo0=bytes.fromhex('ff 02 00 dc 05')#设置初始舵机0位置
        mo1=bytes.fromhex('ff 02 01 82 07')#设置初始舵机1的位置
        mo2 = bytes.fromhex('ff 02 02 dc 05')#设置初始舵机2的位置
        mo3 = bytes.fromhex('ff 02 03 dc 05')#设置初始舵机3的位置
        self.ser.write(m0)
        self.ser.write(m1)
        self.ser.write(m2)
        self.ser.write(mo0)
        self.ser.write(mo1)
        self.ser.write(mo2)
        self.ser.write(mo3)
        
    def simuinit(self):
        return
    
    def xztoq0(self,x,z):
        r=mt.sqrt(x*x+z*z+0.01)
        if z >=0:
            q0=180-mt.acos(z/r)/mt.pi*180
        else:
            q0=mt.acos(abs(z)/r)/mt.pi*180
        return q0
    
    def xytoq12(self,x,y):
        L1=self.ml01.width
        L2=self.ml02.width
        L3=mt.sqrt(y*y+x*x)+0.01
        #L3最长不超过两臂之和
        if L3>L1+L2:
            L3=L1+L2-4
        #L3最短不少于两臂之差
        if L3<abs(L1-L2):
            L3=abs(L1-L2)+4
        p=(L1+L2+L3)/2
        S=mt.sqrt(p*(p-L1)*(p-L2)*(p-L3))
        h=S*2/L3
        Q5=mt.asin(h/L2)/mt.pi*180
        Q4=mt.asin(y/L3)/mt.pi*180
        Q7=mt.asin(h/L1)/mt.pi*180
        Q1=180-Q5-Q7
        Q2=Q5+Q4
        #print("Q1,Q2",Q1,Q2)
        return Q1,Q2
    
    #输入参数为屏幕角度，不考虑电机面对面放置,考虑杆的移动相对位置
    def scrdraw(self,angle0,angle1,angle2):
        #电机0在底盘
        self.m00.drawrelative(angle0)
        #杆02与电机0中心在z轴方向相连
        self.ml02.setzjointpoint(self.m00.center)
        self.ml02.setzangle(angle0)
        #电机02
        self.m02.drawrelative(angle2)
         #杆02,与m02的圆心相连
        self.ml02.setangle(angle2)
        self.ml02.draw()
        self.ml02.setjointpoint(self.m02.center)
        #电机01放在杆02的右端
        self.m01.setcenter(self.ml02.rightend)
        self.m01.zcenter=self.ml02.zrightend
        print("m01 zcenter is",self.m01.zcenter)
        self.m01.startangle=180+self.m02.angle
        if angle1<=0:
            angle1=0
        self.m01.drawrelative(angle1)
        #杆01固定在电机01中心，角度与电机01一样
        self.ml01.setjointpoint(self.m01.center)
        self.ml01.setangle(self.m01.angle)
        self.ml01.setzjointpoint(self.m01.zcenter)
        self.ml01.setzangle(angle0)
        self.ml01.draw()
        self.ml01.drawrightend()
        #杆03永远保持垂直，中心点位于杆01右端下方
        self.ml03.setangle(90)
        L=int(self.ml03.width/2)
        self.ml03.center=np.add(self.ml01.rightend,(0,L))
        self.ml03.setzjointpoint(self.ml01.zrightend)
        self.ml03.setzangle(angle0)
        self.ml03.draw()
        self.ml03.drawrightend()
        #显示串口连接状态红未连接绿连接
        if self.ser is not None:
            cv2.circle(img,(50,50),10,(0,255,0),3)
        else:
            cv2.circle(img,(50,50),10,(0,0,255),3)
        
    #输入角度考虑电机是否对面装配，从而是逆时针旋转，然后画在图上   
    def robodraw(self,angle0,angle1,angle2):
        mangle0=angle0
        mangle1=angle1
        mangle2=angle2
        if self.m00.facetome == True:
            print("facetome")
            mangle0=self.m00.maxangle-angle0
        if self.m01.facetome == True:
            print("facetome")
            mangle1=self.m01.maxangle-angle1
        if self.m02.facetome == True:
            print("facetome")
            mangle2=slef.m02.maxangle-angle2
        print("mangel0,mangle1,mangle2",mangle0,mangle1,mangle2)
        self.scrdraw(mangle0,mangle1,mangle2)
    
    #输入PWM值画机械臂。对于非180度舵机实施缩放处理。对零位非0度的情况进行处理
    def drawpwm(self,pwm0,pwm1,pwm2):
        angle0pc=int((pwm0-500)/2000*180)
        angle0pc=180-angle0pc
        angle1pc=int((pwm1-500)/2000*180)
        angle2pc=int((pwm2-500)/2000*180)
        angel0=int(angle0pc*(self.m00.maxangle-self.m00.minangle)/180)+self.m00.minangle
        angel1=int(angle1pc*(self.m01.maxangle-self.m01.minangle)/180)+self.m01.minangle
        angel2=int(angle2pc*(self.m02.maxangle-self.m02.minangle)/180)+self.m02.minangle
        A1=180+50-(angel1+angel2)
        print("angel0,A1,angel2",angel0,A1,angel2)
        self.robodraw(angel0,A1,angel2)
        
    def xytopwm(self,x,y,z):
        r=mt.sqrt(x*x+z*z)
        q1,q2=self.xytoq12(r,y)
        q2=q2-7.5
        # q1=180+50-(q2+pwmq1)
        #50，20为舵机安装倾斜角度
        pwmq1=180+50-(q1+q2)
        pwmq2=q2
        pwm1=int(pwmq1/180*2000+500)
        pwm2=int(pwmq2/180*2000+500)
        
        q0=self.xztoq0(x,z)
        qq0=q0-self.m00.minangle
        pwmq0=qq0/(self.m00.maxangle-self.m00.minangle)*180
        pwm0=int(pwmq0/180*2000+500)
        print("pwm0,pwm1,pwm2",pwm0,pwm1,pwm2)
        return pwm0,pwm1,pwm2
    
    def serwrite(self,pwm0,pwm1,pwm2):
        if self.ser is None:
            return
        print(pwm0,pwm1,pwm2)
        pw0str="%04x"%pwm0
        pw1str="%04x"%pwm1
        pw2str="%04x"%pwm2
        print(pw0str,pw1str,pw2str)
        mo0hexstr="ff 02 00"+" "+pw0str[2:4]+" "+pw0str[0:2]
        mo1hexstr="ff 02 01"+" "+pw1str[2:4]+" "+pw1str[0:2]
        mo2hexstr="ff 02 02"+" "+pw2str[2:4]+" "+pw2str[0:2]
        self.ser.write(bytes.fromhex(mo0hexstr))
        self.ser.write(bytes.fromhex(mo1hexstr))
        self.ser.write(bytes.fromhex(mo2hexstr))
        return
    
    def robotTwin(self,x,y,z):
        pwm0,pwm1,pwm2=self.xytopwm(x,y,-1*z)
        self.drawpwm(pwm0,pwm1,pwm2)
        self.serwrite(pwm0,pwm1,pwm2)
    def show(self):
        global img
        cv2.imshow("marm",img)
        cv2.waitKey(300)
        #恢复背景
        img=imgcon.copy()
    def home(self):
        self.robotTwin(0,0,0)
        self.show()
        cv2.waitKey(5000)
    def destroy(self):
        if self.ser is not None:
            self.ser.close()
        cv2.destroyAllWindows()
        
#配置零件舵机
mM00=mmotor((170+300,240+300),0,0,-45,225)
mM02=mmotor((170+300,240+300),0,0,0,180)
mM01=mmotor((0,0),0,0,0,180)
mM03=mmotor((0,0),0,0,0,180)
#配置零件梁
mMl01=mlever((0,0),200,10,0)
mMl02=mlever((0,0),200,10,0)
mMl03=mlever((0,0),60,10,0)
#组装成机械臂
marm=Marm(mM00,mM01,mM02,mM03,mMl01,mMl02,mMl03)

marm.home()
for i in range(-20,20):
    marm.robotTwin(300,0,i*10)
    marm.show()    
for i in range(-20,20):
    marm.robotTwin(100,0,i*10)
    marm.show()
for i in range(-20,20):
    marm.robotTwin(50,0,i*10)
    marm.show()
    
marm.home()
for i in range(0,35):
    marm.robotTwin(i*10,0,0)
    marm.show() 
marm.destroy()

# 取切割板右下角一小块区域取得HSV值

In [1]:
import numpy as np
import cv2

img=cv2.imread("cutpad.jpg")
print(len(img),len(img[0]))
height=10
width=10
color=(0,0,0)
imsample=np.ones((height, width, 3), dtype="uint8")
imsample[:] = color
h0=380
w0=500
imsample=img[h0:height+h0,w0:width+w0]
cv2.imshow("imsample",imsample)
cv2.waitKey()

ims_hsv=cv2.cvtColor(imsample,cv2.COLOR_BGR2HSV)
print(ims_hsv)
cv2.imshow("ims_hsv",ims_hsv)
cv2.waitKey()

cv2.destroyAllWindows()

480 640
[[[ 92 125 135]
  [ 91 124 132]
  [ 90 124 125]
  [ 91 126 130]
  [ 92 129 130]
  [ 94 134 137]
  [ 96 135 147]
  [ 97 150 146]
  [ 99 154 156]
  [100 103 253]]

 [[ 94 127 135]
  [ 93 127 131]
  [ 93 127 130]
  [ 93 125 133]
  [ 94 129 134]
  [ 94 133 138]
  [ 96 140 142]
  [ 97 149 147]
  [ 98 149 159]
  [ 99 110 231]]

 [[ 95 131 134]
  [ 94 131 130]
  [ 93 127 131]
  [ 93 124 134]
  [ 92 128 132]
  [ 92 131 134]
  [ 93 144 133]
  [ 94 145 139]
  [ 95 152 141]
  [ 96 130 174]]

 [[ 94 133 134]
  [ 94 128 135]
  [ 94 128 135]
  [ 92 131 128]
  [ 92 132 131]
  [ 92 130 135]
  [ 91 138 131]
  [ 92 139 136]
  [ 94 155 127]
  [ 94 153 133]]

 [[ 94 130 133]
  [ 93 129 138]
  [ 93 137 130]
  [ 93 135 132]
  [ 93 136 133]
  [ 92 134 131]
  [ 92 134 131]
  [ 92 134 131]
  [ 93 136 133]
  [ 93 134 133]]

 [[ 93 134 133]
  [ 93 131 136]
  [ 93 139 130]
  [ 93 137 132]
  [ 93 136 133]
  [ 93 136 133]
  [ 92 132 131]
  [ 92 132 131]
  [ 92 128 131]
  [ 92 128 131]]

 [[ 93 134 133]
  [ 

# 给绿色的高低定门限值找到图像中的绿色板底

In [2]:
import numpy as np
import cv2

img=cv2.imread("cutpad.jpg")

img_hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
print(img_hsv)
cv2.imshow("ims_hsv",img_hsv)
cv2.waitKey()

lower_green = np.array([76,80,0])
upper_green = np.array([105,180,255])

mask = cv2.inRange(img_hsv, lower_green, upper_green)
cv2.imshow("mask",mask)
cv2.waitKey()

cv2.destroyAllWindows()

[[[ 86  91  70]
  [ 86  75  85]
  [ 86  67  84]
  ...
  [100  95  89]
  [ 98 100  79]
  [ 98 103  77]]

 [[ 86  90  71]
  [ 86  73  87]
  [ 86  66  85]
  ...
  [100  95  89]
  [ 98 101  78]
  [ 98 103  77]]

 [[ 86  87  73]
  [ 86  72  88]
  [ 86  65  86]
  ...
  [100  97  87]
  [100 107  79]
  [100 108  78]]

 ...

 [[ 50  46 111]
  [ 50  47 109]
  [ 50  47 108]
  ...
  [ 97 139  46]
  [ 97 139  46]
  [ 97 139  46]]

 [[ 54  42 116]
  [ 54  42 114]
  [ 51  47 113]
  ...
  [ 96 144  46]
  [ 96 144  46]
  [ 96 147  45]]

 [[ 54  44 111]
  [ 54  44 110]
  [ 51  49 109]
  ...
  [ 96 144  46]
  [ 96 147  45]
  [ 96 151  44]]]


In [ ]:
利用膨胀进一步让棋盘区域连通，找到大型轮廓，得到矩形区域

In [3]:
import numpy as np
import cv2

KERNEL = np.ones((5,5),np.uint8)

img=cv2.imread("cutpad.jpg")
imgcon=img.copy()

img_hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
cv2.imshow("ims_hsv",img_hsv)
cv2.waitKey()

lower_green = np.array([86,80,100])
upper_green = np.array([95,180,255])

mask = cv2.inRange(img_hsv, lower_green, upper_green)
cv2.imshow("mask",mask)
cv2.waitKey()

dilation = cv2.dilate(mask,KERNEL,iterations = 2)
cv2.imshow("dilation",dilation)
cv2.waitKey

conts, hier = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#print("cont",conts)
#print("hier",hier)

maxi=0
for i in range(len(conts)):
    area = cv2.contourArea(conts[i])
    if area >10000:
        print("area",area)
        img=cv2.drawContours(img,conts,i,(100,100,255),3)
        maxi=i

cv2.imshow("dilationimg",img)
cv2.waitKey()

x,y,w,h=cv2.boundingRect(conts[maxi])
cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
cv2.imshow("dilationimg",img)
cv2.waitKey()

impad=np.ones((h, w, 3), dtype="uint8")
impad=imgcon[y:y+h,x:x+w]
cv2.imshow("impad",impad)
cv2.waitKey()


cv2.destroyAllWindows()

area 189400.5


# 与机械臂仿真软件集成在一起

In [ ]:
import numpy as np
import cv2
import math  as mt
import serial
import serial.tools.list_ports
import string
import binascii
import time

KERNEL = np.ones((5,5),np.uint8)

img=cv2.imread("cutpad.jpg")
imgcon=img.copy()

img_hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
cv2.imshow("ims_hsv",img_hsv)
cv2.waitKey()

lower_green = np.array([86,80,100])
upper_green = np.array([95,180,255])

mask = cv2.inRange(img_hsv, lower_green, upper_green)
cv2.imshow("mask",mask)
cv2.waitKey()

dilation = cv2.dilate(mask,KERNEL,iterations = 2)
cv2.imshow("dilation",dilation)
cv2.waitKey

conts, hier = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

maxi=0
for i in range(len(conts)):
    area = cv2.contourArea(conts[i])
    if area >10000:
        print("area",area)
        img=cv2.drawContours(img,conts,i,(100,100,255),3)
        maxi=i

cv2.imshow("dilationimg",img)
cv2.waitKey()

x,y,w,h=cv2.boundingRect(conts[maxi])
cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
cv2.imshow("dilationimg",img)
cv2.waitKey()
impad=np.ones((h, w, 3), dtype="uint8")
impad=imgcon[y:y+h,x:x+w]
cv2.imshow("impad",impad)
cv2.waitKey()
cv2.destroyAllWindows()

img=cv2.imread("marm.jpg")
img9sea=impad
#img9sea=cv2.flip(img9sea,0)
#img9sea=cv2.flip(img9sea,1)
img9sea=cv2.resize(img9sea,(590,460))
#图像缩一半便于屏幕显示处理
shape=img.shape
print(shape)
H=int(shape[0]/3)
W=int(shape[1]/3)
print(W,H)

shape2=img9sea.shape
print(shape2)
H2=shape2[0]
W2=shape2[1]

img=cv2.resize(img,(W,H))
img= cv2.copyMakeBorder(img,300,100,300,100,cv2.BORDER_CONSTANT,value=(100,100,0))
img[140:140+H2,190:190+W2]=img9sea

imgcon=img.copy()

#顶视图平面中点
zcenter=(470,300)

#电机模型,
class mmotor():
    def __init__(self,center,angle,startangle,minangle,maxangle):
        self.center=center
        self.angle=angle
        #电机中心点随杠杆旋转时电机0位与水平线的夹角
        self.startangle=startangle
        #电机中点为90度，顺时针方向为正，最小点和最大点角度
        self.maxangle=maxangle
        self.minangle=minangle
        self.relaangle=0
        self.facetome=False
        self.zcenter=(0,0)
        
    def setcenter(self,center):
        self.center=center
        
    def draw(self,angle):
        self.angle=angle
        #通过舵机角度270判断是否为底座舵机，画图时大一些
        if self.maxangle-self.startangle >200:
            r=50
            color=(0,255,0)
            cv2.circle(img,self.center,r,color,3)
            rect=(self.center,(2*r,4),angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            cv2.drawContours(img,[box],0,color,3)
        else:
            r=20
            color=(155,0,0)
            cv2.circle(img,self.center,r,color,3)
            rect=(self.center,(2*r,4),angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            cv2.drawContours(img,[box],0,color,3)
      
    def drawrelative(self,relaangle):
        if relaangle >self.maxangle:
            relaangle=self.maxangle
        self.relaangle=relaangle
        self.angle=self.startangle+self.relaangle
        self.draw(self.angle)

# 臂模型
class mlever():
    def __init__(self,center,width,high,angle):
        self.center=center
        self.width=width
        self.angle=angle
        self.height=high
        self.jointpoint=(0,0)
        self.zjointpoint=(0,0)
        self.rightend=(width,0)
        self.zrightend=(0,0)
        self.rightendr=10
        self.zcenter=(0,0)
        self.zwidth=100
        self.zheight=10
        self.zangle=60
    
    def draw(self):
        global img,imgcon
        height=100
        # 没有连接点，以中心为轴旋转
        if self.jointpoint != (0,0):
            # 有连接点，则重新确定中心，绕连接点转 w,h为中心点相对坐标
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            h=int(self.width/2*mt.sin(self.angle/180*mt.pi))
            #z轴的投影长度为中心点坐标的一倍
            self.zwidth=2*w
            self.center=(self.jointpoint[0]-w,self.jointpoint[1]-h)
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(self.jointpoint[0]-w*2,self.jointpoint[1]-h*2)
        else:
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            #z轴的投影长度为中心点坐标的一倍
            self.zwidth=2*w
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(int((box[2][0]+box[3][0])/2),int((box[2][1]+box[3][1])/2))
        cv2.drawContours(img,[box],0,(155,0,0),3)
        #画Z轴的投影
        print("ZJ point",self.zjointpoint)
        if (self.zwidth==0):
            zwidth=75
        else:
            zwidth=self.zwidth
        w=int(zwidth/2*mt.cos(self.zangle/180*mt.pi))
        h=int(zwidth/2*mt.sin(self.zangle/180*mt.pi))
        self.zcenter=(self.zjointpoint[0]-w,self.zjointpoint[1]-h)
 
        zrect=(self.zcenter, (zwidth,self.zheight), self.zangle)
        print("zrect",zrect)
        zbox = cv2.boxPoints(zrect).astype(np.int0)
        self.zrightend=(int((zbox[0][0]+zbox[1][0])/2),int((zbox[0][1]+zbox[1][1])/2))
        cv2.drawContours(img,[zbox],0,(0,255,0),3)
        
    def setl(self,center,angle):
        self.center=center
        self.angle=angle
    def setangle(self,angle):
        self.angle=angle    
    def setjointpoint(self,jointpoint):
        self.jointpoint=jointpoint
    def setzjointpoint(self,jointpoint):
        self.zjointpoint=jointpoint
    def setzangle(self,angle):
        self.zangle=angle
    def drawrightend(self):
        cv2.circle(img,self.rightend,self.rightendr,(0,0,255),3)
        cv2.circle(img,self.zrightend,self.rightendr,(0,255,0),3)

class Marm():
    def __init__(self,m00,m01,m02,m03,ml01,ml02,ml03):
        self.m00=m00
        self.m01=m01
        self.m02=m02
        self.m03=m03        
        self.ml01=ml01
        self.ml02=ml02
        self.ml03=ml03 
        self.ser=None
        self.img=None
        self.imgcon=None
        self.zl03=75
        #self.x0>=self.zl03
        self.x0=self.zl03
        self.y0=0
        self.xmm=0
        self.ymm=0
        self.zmm=0
        #模拟器初始化
        self.simuinit()
        #串口及机械臂位置初始化
        self.robotinit()
        
    def robotinit(self):
        #打开串口
        ports = list(serial.tools.list_ports.comports())
        print (ports)
        #便利 寻找串口
        for p in ports:
            print (p[1])
            if "serial" in p[1] or "CH340" in p[1]:
                self.ser=serial.Serial(port=p[0])
                if self.ser is not None:
                    print(self.ser)
                    print("serial conected")
                else:
                    print("serial not connected")
                    return
            else:
                print ("No CH340 Device was found connected to the computer")
                return
        if self.ser is None:
            return
        m0=bytes.fromhex('ff 01 00 08 00')# 舵机0速度为 72
        m1=bytes.fromhex('ff 01 01 08 00')# 舵机1速度为 72
        m2=bytes.fromhex('ff 01 02 08 00')# 舵机2速度为 72
        mo0=bytes.fromhex('ff 02 00 dc 05')#设置初始舵机0位置
        mo1=bytes.fromhex('ff 02 01 82 07')#设置初始舵机1的位置
        mo2 = bytes.fromhex('ff 02 02 dc 05')#设置初始舵机2的位置
        mo3 = bytes.fromhex('ff 02 03 dc 05')#设置初始舵机3的位置
        self.ser.write(m0)
        self.ser.write(m1)
        self.ser.write(m2)
        self.ser.write(mo0)
        self.ser.write(mo1)
        self.ser.write(mo2)
        self.ser.write(mo3)
        
    def simuinit(self):
        return
    
    def xztoq0(self,x,z):
        r=mt.sqrt(x*x+z*z+0.01)
        if z >=0:
            q0=180-mt.acos(z/r)/mt.pi*180
        else:
            q0=mt.acos(abs(z)/r)/mt.pi*180
        print("xztoq0,q0 is",q0)
        return q0
    
    def xytoq12(self,x,y):
        L1=self.ml01.width
        L2=self.ml02.width
        L3=mt.sqrt(y*y+x*x)+0.001
        #L3最长不超过两臂之和
        if L3>L1+L2:
            L3=L1+L2-4
        #L3最短不少于两臂之差
        if L3<abs(L1-L2):
            L3=abs(L1-L2)+4
        p=(L1+L2+L3)/2
        S=mt.sqrt(p*(p-L1)*(p-L2)*(p-L3))
        h=S*2/L3
        Q5=mt.asin(h/L2)/mt.pi*180
        Q4=mt.asin(y/L3)/mt.pi*180
        Q7=mt.asin(h/L1)/mt.pi*180
        Q1=180-Q5-Q7
        Q2=Q5+Q4
        #print("Q1,Q2",Q1,Q2)
        return Q1,Q2
    
    #输入参数为屏幕角度，不考虑电机面对面放置,考虑杆的移动相对位置
    def scrdraw(self,angle0,angle1,angle2):
        #电机0在底盘,安装误差10度
        angle0+=10
        self.m00.drawrelative(angle0)
        #杆02与电机0中心在z轴方向相连
        self.ml02.setzjointpoint(self.m00.center)
        self.ml02.setzangle(angle0)
        #电机02
        self.m02.drawrelative(angle2)
         #杆02,与m02的圆心相连
        self.ml02.setangle(angle2)
        self.ml02.draw()
        self.ml02.setjointpoint(self.m02.center)
        #电机01放在杆02的右端
        self.m01.setcenter(self.ml02.rightend)
        self.m01.zcenter=self.ml02.zrightend
        print("m01 zcenter is",self.m01.zcenter)
        self.m01.startangle=180+self.m02.angle
        if angle1<=0:
            angle1=0
        self.m01.drawrelative(angle1)
        #杆01固定在电机01中心，角度与电机01一样
        self.ml01.setjointpoint(self.m01.center)
        self.ml01.setangle(self.m01.angle)
        self.ml01.setzjointpoint(self.m01.zcenter)
        self.ml01.setzangle(angle0)
        self.ml01.draw()
        self.ml01.drawrightend()
        #杆03永远保持垂直，中心点位于杆01右端下方靠左75
        self.ml03.setangle(90)
        L=int(self.ml03.width/2)
        self.ml03.center=np.add(self.ml01.rightend,(-self.zl03,L))
        self.ml03.setzjointpoint(self.ml01.zrightend)
        self.ml03.setzangle(angle0)
        self.ml03.draw()
        self.ml03.drawrightend()
        #显示串口连接状态红未连接绿连接
        if self.ser is not None:
            cv2.circle(img,(50,50),10,(0,255,0),3)
        else:
            cv2.circle(img,(50,50),10,(0,0,255),3)
        
    #输入角度考虑电机是否对面装配，从而是逆时针旋转，然后画在图上   
    def robodraw(self,angle0,angle1,angle2):
        mangle0=angle0
        mangle1=angle1
        mangle2=angle2
        if self.m00.facetome == True:
            print("facetome")
            mangle0=self.m00.maxangle-angle0
        if self.m01.facetome == True:
            print("facetome")
            mangle1=self.m01.maxangle-angle1
        if self.m02.facetome == True:
            print("facetome")
            mangle2=slef.m02.maxangle-angle2
        print("mangel0,mangle1,mangle2",mangle0,mangle1,mangle2)
        self.scrdraw(mangle0,mangle1,mangle2)
    
    #输入PWM值画机械臂。对于非180度舵机实施缩放处理。对零位非0度的情况进行处理
    def drawpwm(self,pwm0,pwm1,pwm2):
        angle0pc=int((pwm0-500)/2000*180)
        angle0pc=180-angle0pc
        angle1pc=int((pwm1-500)/2000*180)
        angle2pc=int((pwm2-500)/2000*180)
        angel0=int(angle0pc*(self.m00.maxangle-self.m00.minangle)/180)+self.m00.minangle
        angel1=int(angle1pc*(self.m01.maxangle-self.m01.minangle)/180)+self.m01.minangle
        angel2=int(angle2pc*(self.m02.maxangle-self.m02.minangle)/180)+self.m02.minangle
        A1=180+50-(angel1+angel2)
        print("angel0,A1,angel2",angel0,A1,angel2)
        self.robodraw(angel0,A1,angel2)
        
    def xytopwm(self,x,y,z):
        rx=mt.sqrt((x+self.x0)*(x+self.x0)+z*z)
        r=rx-self.x0
        q1,q2=self.xytoq12(r,y+self.y0)
        q2=q2-4
        # q1=180+50-(q2+pwmq1)
        #50，20为舵机安装倾斜角度
        pwmq1=180+55-(q1+q2)
        pwmq2=q2
        pwm1=int(pwmq1/180*2000+500)
        pwm2=int(pwmq2/180*2000+500)
        
        q0=self.xztoq0(x+self.x0,z)
        qq0=q0-self.m00.minangle
        pwmq0=qq0/(self.m00.maxangle-self.m00.minangle)*180
        pwm0=int(pwmq0/180*2000+500)
        print("pwm0,pwm1,pwm2",pwm0,pwm1,pwm2)
        return pwm0,pwm1,pwm2
    
    def serwrite(self,pwm0,pwm1,pwm2):
        if self.ser is None:
            return
        print(pwm0,pwm1,pwm2)
        pw0str="%04x"%pwm0
        pw1str="%04x"%pwm1
        pw2str="%04x"%pwm2
        print(pw0str,pw1str,pw2str)
        mo0hexstr="ff 02 00"+" "+pw0str[2:4]+" "+pw0str[0:2]
        mo1hexstr="ff 02 01"+" "+pw1str[2:4]+" "+pw1str[0:2]
        mo2hexstr="ff 02 02"+" "+pw2str[2:4]+" "+pw2str[0:2]
        self.ser.write(bytes.fromhex(mo0hexstr))
        self.ser.write(bytes.fromhex(mo1hexstr))
        self.ser.write(bytes.fromhex(mo2hexstr))
        return
    
    def robotTwin(self,x,y,z):
        print("robotTwin x y z",x,y,z)
        pwm0,pwm1,pwm2=self.xytopwm(x,y,-1*z)
        self.drawpwm(pwm0,pwm1,pwm2)
        self.serwrite(pwm0,pwm1,pwm2)
    
    def robotTwinMM(self,xmm,ymm,zmm):
        self.xmm=xmm
        self.ymm=ymm
        self.zmm=zmm
        x=xmm*200/130
        #补偿越远离缝隙越大
        y=(ymm-0.08*xmm)*200/120
        z=zmm*100/71/(1+1/(xmm/20+3))-1
        self.robotTwin(x,y,z)
        return
    def pumpOn(self,nsec):
        #小于0的值表示一直有气泵
        if self.ser is None:
            return
        mo4 = bytes.fromhex('ff 02 04 c4 09')#设置初始舵机4的位置.180为气泵开
        self.ser.write(mo4)
        time.sleep(nsec)
     
    def pumpOff(self):
        if self.ser is None:
            return
        mo4 = bytes.fromhex('ff 02 04 00 00')#设置初始舵机4的位置.0为气泵关
        self.ser.write(mo4)   
    
    def valve(self,onoff):
        if self.ser is None:
            return
        if onoff == 1:
            mo5=bytes.fromhex('ff 02 05 c4 09')#设置初始舵机5的位置.180为气阀导通，0为气阀阻塞
        else:
            mo5=bytes.fromhex('ff 02 05 00 00')#设置初始舵机5的位置.180为气阀导通，0为气阀阻塞
        self.ser.write(mo5)
    
    def moveto(self,xmm,ymm,zmm):
        #从当前位置抬起
        self.robotTwinMM(self.xmm,self.ymm+50,self.zmm)
        self.show()
        time.sleep(0.5)
        #从空中移动到目的地上方
        self.robotTwinMM(xmm,self.ymm,zmm)
        self.show()
        time.sleep(0.5)
        #到目的地
        self.robotTwinMM(xmm,ymm,zmm)
        self.show()
        time.sleep(0.5)
        
    def show(self):
        global img
        cv2.imshow("marm",img)
        cv2.waitKey(300)
        #恢复背景
        img=imgcon.copy()
    def home(self):
        self.robotTwin(0,0,0)
        self.pumpOn(1)
        self.valve(0)
        time.sleep(3)
        self.pumpOff()
        self.valve(1)
        self.show()
        cv2.waitKey(5000)
    def destroy(self):
        if self.ser is not None:
            self.ser.close()
        cv2.destroyAllWindows()


def ChessBoard():
    return
        
#配置零件舵机
mM00=mmotor((170+300,240+300),0,0,-70,240)
mM02=mmotor((170+300,240+300),0,0,0,180)
mM01=mmotor((0,0),0,0,0,180)
mM03=mmotor((0,0),0,0,0,180)
#配置零件梁
mMl01=mlever((0,0),200,10,0)
mMl02=mlever((0,0),200,10,0)
mMl03=mlever((0,0),60,10,0)
#组装成机械臂
marm=Marm(mM00,mM01,mM02,mM03,mMl01,mMl02,mMl03)


marm.home()
time.sleep(5)
marm.robotTwinMM(100,0,-200)
marm.show()
time.sleep(3)
marm.robotTwinMM(100,50,-200)
marm.show()
time.sleep(3)
marm.robotTwinMM(100,50,200)
marm.show()
time.sleep(3)
marm.robotTwinMM(100,0,200)
marm.show()
time.sleep(3)

points=[(50,0,0),(50,0,-150),(150,0,-150),(150,0,0),(150,0,150),(50,0,150)]

while True:
    #ret,frame=cv2.Video
    marm.home()
    marm.show()
    time.sleep(5)
    i=0
    for point in points:
        i+=1
        marm.moveto(point[0],point[1],point[2])
        marm.valve(0)
        marm.pumpOn(3)
        marm.pumpOff()
        marm.moveto(200,20+i*2,200)
        marm.pumpOff()
        marm.valve(1)
        marm.moveto(200,20+i*2,200)
        time.sleep(6)
marm.destroy()

area 189400.5
(1080, 1440, 3)
480 360
(460, 590, 3)
USB-SERIAL CH340 (COM7)
Serial<id=0x23f50540898, open=True>(port='COM7', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)
serial conected
robotTwin x y z 0 0 0
xztoq0,q0 is 90.0
pwm0,pwm1,pwm2 1532 2155 1455
angel0,A1,angel2 81 -3 85
mangel0,mangle1,mangle2 81 -3 85
ZJ point (470, 540)
zrect ((470, 533), (16, 10), 91)
m01 zcenter is (470, 524)
ZJ point (470, 524)
zrect ((470, 531), (-16, 10), 91)
ZJ point (469, 538)
zrect ((469, 501), (75, 10), 91)
1532 2155 1455
05fc 086b 05af
robotTwin x y z 153.84615384615384 -13.333333333333334 -251.39123630672924
xztoq0,q0 is 137.68781459314667
pwm0,pwm1,pwm2 1839 1725 961
angel0,A1,angel2 33 79 41
mangel0,mangle1,mangle2 33 79 41
ZJ point (470, 540)
zrect ((416, 489), (150, 10), 43)
m01 zcenter is (360, 437)
ZJ point (360, 437)
zrect ((324, 403), (100, 10), 43)
ZJ point (287, 368)
zrect ((260, 343), (75, 10), 43)
1839 1725 961
072f 06bd 

robotTwin x y z 307.6923076923077 13.333333333333334 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1288 632
angel0,A1,angel2 116 149 11
mangel0,mangle1,mangle2 116 149 11
ZJ point (470, 540)
zrect ((527, 461), (196, 10), 126)
m01 zcenter is (584, 381)
ZJ point (584, 381)
zrect ((638, 306), (186, 10), 126)
ZJ point (692, 230)
zrect ((714, 200), (75, 10), 126)
1311 1288 632
051f 0508 0278
robotTwin x y z 307.6923076923077 96.66666666666667 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1018 628
angel0,A1,angel2 116 173 11
mangel0,mangle1,mangle2 116 173 11
ZJ point (470, 540)
zrect ((527, 461), (196, 10), 126)
m01 zcenter is (584, 381)
ZJ point (584, 381)
zrect ((642, 301), (198, 10), 126)
ZJ point (700, 220)
zrect ((722, 190), (75, 10), 126)
1311 1018 628
051f 03fa 0274
robotTwin x y z 307.6923076923077 96.66666666666667 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1018 628
angel0,A1,angel2 116 173 11
mangel0,mangle1,mang

robotTwin x y z 307.6923076923077 20.0 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1275 641
angel0,A1,angel2 116 149 12
mangel0,mangle1,mangle2 116 149 12
ZJ point (470, 540)
zrect ((527, 462), (194, 10), 126)
m01 zcenter is (583, 383)
ZJ point (583, 383)
zrect ((638, 307), (188, 10), 126)
ZJ point (693, 230)
zrect ((715, 200), (75, 10), 126)
1311 1275 641
051f 04fb 0281
robotTwin x y z 307.6923076923077 103.33333333333333 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1077 713
angel0,A1,angel2 116 160 19
mangel0,mangle1,mangle2 116 160 19
ZJ point (470, 540)
zrect ((525, 464), (188, 10), 126)
m01 zcenter is (580, 387)
ZJ point (580, 387)
zrect ((638, 307), (198, 10), 126)
ZJ point (696, 226)
zrect ((718, 196), (75, 10), 126)
1311 1077 713
051f 0435 02c9
robotTwin x y z 230.76923076923077 110.0 191.89650949173298
xztoq0,q0 is 57.88819218284143
pwm0,pwm1,pwm2 1325 1366 1133
angel0,A1,angel2 112 97 56
mangel0,mangle1,mangle2 112 97 56
ZJ point

robotTwin x y z 76.92307692307692 130.0 -1.0
xztoq0,q0 is 90.37713124832872
pwm0,pwm1,pwm2 1534 1249 1868
angel0,A1,angel2 79 40 123
mangel0,mangle1,mangle2 79 40 123
ZJ point (470, 540)
zrect ((470, 593), (-108, 10), 89)
m01 zcenter is (470, 646)
ZJ point (470, 646)
zrect ((469, 552), (190, 10), 89)
ZJ point (467, 456)
zrect ((467, 419), (75, 10), 89)
1534 1249 1868
05fe 04e1 074c
robotTwin x y z 76.92307692307692 -6.666666666666667 -1.0
xztoq0,q0 is 90.37713124832872
pwm0,pwm1,pwm2 1534 2086 1276
angel0,A1,angel2 79 19 69
mangel0,mangle1,mangle2 79 19 69
ZJ point (470, 540)
zrect ((470, 506), (70, 10), 89)
m01 zcenter is (469, 470)
ZJ point (469, 470)
zrect ((469, 472), (-6, 10), 89)
ZJ point (469, 474)
zrect ((469, 437), (75, 10), 89)
1534 2086 1276
05fe 0826 04fc
robotTwin x y z 76.92307692307692 76.66666666666667 -1.0
xztoq0,q0 is 90.37713124832872
pwm0,pwm1,pwm2 1534 1481 1779
angel0,A1,angel2 79 27 115
mangel0,mangle1,mangle2 79 27 115
ZJ point (470, 540)
zrect ((470, 581), (-84

robotTwin x y z 307.6923076923077 56.666666666666664 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1189 674
angel0,A1,angel2 116 153 15
mangel0,mangle1,mangle2 116 153 15
ZJ point (470, 540)
zrect ((526, 463), (192, 10), 126)
m01 zcenter is (582, 385)
ZJ point (582, 385)
zrect ((639, 307), (194, 10), 126)
ZJ point (695, 228)
zrect ((717, 198), (75, 10), 126)
1311 1189 674
051f 04a5 02a2
robotTwin x y z 307.6923076923077 16.666666666666668 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1282 636
angel0,A1,angel2 116 148 12
mangel0,mangle1,mangle2 116 148 12
ZJ point (470, 540)
zrect ((527, 462), (194, 10), 126)
m01 zcenter is (583, 383)
ZJ point (583, 383)
zrect ((637, 308), (186, 10), 126)
ZJ point (691, 232)
zrect ((713, 202), (75, 10), 126)
1311 1282 636
051f 0502 027c
robotTwin x y z 307.6923076923077 100.0 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1083 708
angel0,A1,angel2 116 160 18
mangel0,mangle1,mangle2 116 160

robotTwin x y z 307.6923076923077 106.66666666666667 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1072 719
angel0,A1,angel2 116 160 19
mangel0,mangle1,mangle2 116 160 19
ZJ point (470, 540)
zrect ((525, 464), (188, 10), 126)
m01 zcenter is (580, 387)
ZJ point (580, 387)
zrect ((638, 307), (198, 10), 126)
ZJ point (696, 226)
zrect ((718, 196), (75, 10), 126)
1311 1072 719
051f 0430 02cf
robotTwin x y z 307.6923076923077 23.333333333333332 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1269 645
angel0,A1,angel2 116 148 13
mangel0,mangle1,mangle2 116 148 13
ZJ point (470, 540)
zrect ((527, 462), (194, 10), 126)
m01 zcenter is (583, 383)
ZJ point (583, 383)
zrect ((638, 307), (188, 10), 126)
ZJ point (693, 230)
zrect ((715, 200), (75, 10), 126)
1311 1269 645
051f 04f5 0285
robotTwin x y z 307.6923076923077 106.66666666666667 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1072 719
angel0,A1,angel2 116 160 19
mangel0,mangle1,ma

robotTwin x y z 307.6923076923077 93.33333333333333 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1049 649
angel0,A1,angel2 116 168 13
mangel0,mangle1,mangle2 116 168 13
ZJ point (470, 540)
zrect ((527, 462), (194, 10), 126)
m01 zcenter is (583, 383)
ZJ point (583, 383)
zrect ((641, 303), (198, 10), 126)
ZJ point (699, 222)
zrect ((721, 192), (75, 10), 126)
1311 1049 649
051f 0419 0289
robotTwin x y z 76.92307692307692 113.33333333333333 -179.76489707475622
xztoq0,q0 is 139.79814182330804
pwm0,pwm1,pwm2 1853 1437 1520
angel0,A1,angel2 31 55 91
mangel0,mangle1,mangle2 31 55 91
ZJ point (470, 540)
zrect ((470, 540), (-2, 10), 41)
m01 zcenter is (470, 540)
ZJ point (470, 540)
zrect ((409, 487), (164, 10), 41)
ZJ point (346, 432)
zrect ((318, 408), (75, 10), 41)
1853 1437 1520
073d 059d 05f0
robotTwin x y z 76.92307692307692 -6.666666666666667 -179.76489707475622
xztoq0,q0 is 139.79814182330804
pwm0,pwm1,pwm2 1853 1919 1166
angel0,A1,angel2 31 44 59
mangel0,mangle1,ma

robotTwin x y z 230.76923076923077 63.333333333333336 -1.0
xztoq0,q0 is 90.18738174548115
pwm0,pwm1,pwm2 1533 1576 1217
angel0,A1,angel2 81 70 64
mangel0,mangle1,mangle2 81 70 64
ZJ point (470, 540)
zrect ((470, 498), (86, 10), 91)
m01 zcenter is (470, 454)
ZJ point (470, 454)
zrect ((471, 386), (138, 10), 91)
ZJ point (472, 316)
zrect ((472, 279), (75, 10), 91)
1533 1576 1217
05fd 0628 04c1
robotTwin x y z 307.6923076923077 56.666666666666664 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1189 674
angel0,A1,angel2 116 153 15
mangel0,mangle1,mangle2 116 153 15
ZJ point (470, 540)
zrect ((526, 463), (192, 10), 126)
m01 zcenter is (582, 385)
ZJ point (582, 385)
zrect ((639, 307), (194, 10), 126)
ZJ point (695, 228)
zrect ((717, 198), (75, 10), 126)
1311 1189 674
051f 04a5 02a2
robotTwin x y z 307.6923076923077 20.0 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1275 641
angel0,A1,angel2 116 149 12
mangel0,mangle1,mangle2 116 149 12
ZJ point (470,

robotTwin x y z 307.6923076923077 110.0 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1066 724
angel0,A1,angel2 116 160 20
mangel0,mangle1,mangle2 116 160 20
ZJ point (470, 540)
zrect ((524, 465), (186, 10), 126)
m01 zcenter is (578, 389)
ZJ point (578, 389)
zrect ((636, 309), (200, 10), 126)
ZJ point (694, 228)
zrect ((716, 198), (75, 10), 126)
1311 1066 724
051f 042a 02d4
robotTwin x y z 307.6923076923077 110.0 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1066 724
angel0,A1,angel2 116 160 20
mangel0,mangle1,mangle2 116 160 20
ZJ point (470, 540)
zrect ((524, 465), (186, 10), 126)
m01 zcenter is (578, 389)
ZJ point (578, 389)
zrect ((636, 309), (200, 10), 126)
ZJ point (694, 228)
zrect ((716, 198), (75, 10), 126)
1311 1066 724
051f 042a 02d4
robotTwin x y z 307.6923076923077 26.666666666666668 260.569416498994
xztoq0,q0 is 55.74964300526441
pwm0,pwm1,pwm2 1311 1262 649
angel0,A1,angel2 116 149 13
mangel0,mangle1,mangle2 116 149 13
ZJ point 

# 视频集成

In [1]:
import numpy as np
import cv2
import math  as mt
import serial
import serial.tools.list_ports
import string
import binascii
import time

img=cv2.imread("cutpad.jpg")
cap = cv2.VideoCapture(0)
xcon=0
ycon=0
wcon=0
hcon=0

def getimgcon():
    global cap
    global xcon,ycon,wcon,hcon
    ret, frame = cap.read()
    KERNEL = np.ones((5,5),np.uint8)
    img_hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
#    cv2.imshow("ims_hsv",img_hsv)
#    cv2.waitKey()
    
    lower_green = np.array([86,80,100])
    upper_green = np.array([95,180,255])
    
    mask = cv2.inRange(img_hsv, lower_green, upper_green)
#    cv2.imshow("mask",mask)
#    cv2.waitKey()
    
    dilation = cv2.dilate(mask,KERNEL,iterations = 2)
#    cv2.imshow("dilation",dilation)
#    cv2.waitKey()
    
    conts, hier = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    maxi=0
    for i in range(len(conts)):
        area = cv2.contourArea(conts[i])
        if area >10000:
            print("area",area)
 #           frame=cv2.drawContours(frame,conts,i,(100,100,255),3)
            maxi=i
    
 #   cv2.imshow("dilationimg",frame)
 #   cv2.waitKey()
    if wcon ==0 and hcon ==0:
        xcon,ycon,wcon,hcon=cv2.boundingRect(conts[maxi])
#    cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
#    cv2.imshow("dilationimg",frame)
#    cv2.waitKey()
    impad=np.ones((hcon, wcon, 3), dtype="uint8")
    impad=frame[ycon:ycon+hcon,xcon:xcon+wcon]
#    cv2.imshow("impad",impad)
#    cv2.waitKey()
#    cv2.destroyAllWindows()


    img1=cv2.imread("marm.jpg")
    img9sea=impad
    img9sea=cv2.resize(img9sea,(590,460))
    #图像缩一半便于屏幕显示处理
    shape=img1.shape
#    print(shape)
    H=int(shape[0]/3)
    W=int(shape[1]/3)
#print(W,H)
    shape2=img9sea.shape
#print(shape2)
    H2=shape2[0]
    W2=shape2[1]

    img1=cv2.resize(img1,(W,H))
    img1= cv2.copyMakeBorder(img1,300,100,300,100,cv2.BORDER_CONSTANT,value=(100,100,0))
    img1[140:140+H2,190:190+W2]=img9sea
    imgcon1=img1.copy()
    return imgcon1


#顶视图平面中点
zcenter=(470,300)

#电机模型,
class mmotor():
    def __init__(self,center,angle,startangle,minangle,maxangle):
        self.center=center
        self.angle=angle
        #电机中心点随杠杆旋转时电机0位与水平线的夹角
        self.startangle=startangle
        #电机中点为90度，顺时针方向为正，最小点和最大点角度
        self.maxangle=maxangle
        self.minangle=minangle
        self.relaangle=0
        self.facetome=False
        self.zcenter=(0,0)
        
    def setcenter(self,center):
        self.center=center
        
    def draw(self,angle):
        self.angle=angle
        #通过舵机角度270判断是否为底座舵机，画图时大一些
        if self.maxangle-self.startangle >200:
            r=50
            color=(0,255,0)
            cv2.circle(img,self.center,r,color,3)
            rect=(self.center,(2*r,4),angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            cv2.drawContours(img,[box],0,color,3)
        else:
            r=20
            color=(155,0,0)
            cv2.circle(img,self.center,r,color,3)
            rect=(self.center,(2*r,4),angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            cv2.drawContours(img,[box],0,color,3)
      
    def drawrelative(self,relaangle):
        if relaangle >self.maxangle:
            relaangle=self.maxangle
        self.relaangle=relaangle
        self.angle=self.startangle+self.relaangle
        self.draw(self.angle)

# 臂模型
class mlever():
    def __init__(self,center,width,high,angle):
        self.center=center
        self.width=width
        self.angle=angle
        self.height=high
        self.jointpoint=(0,0)
        self.zjointpoint=(0,0)
        self.rightend=(width,0)
        self.zrightend=(0,0)
        self.rightendr=10
        self.zcenter=(0,0)
        self.zwidth=100
        self.zheight=10
        self.zangle=60
    
    def draw(self):
        global img,imgcon
        height=100
        # 没有连接点，以中心为轴旋转
        if self.jointpoint != (0,0):
            # 有连接点，则重新确定中心，绕连接点转 w,h为中心点相对坐标
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            h=int(self.width/2*mt.sin(self.angle/180*mt.pi))
            #z轴的投影长度为中心点坐标的一倍
            self.zwidth=2*w
            self.center=(self.jointpoint[0]-w,self.jointpoint[1]-h)
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(self.jointpoint[0]-w*2,self.jointpoint[1]-h*2)
        else:
            w=int(self.width/2*mt.cos(self.angle/180*mt.pi))
            #z轴的投影长度为中心点坐标的一倍
            self.zwidth=2*w
            rect=(self.center, (self.width,self.height), self.angle)
            box = cv2.boxPoints(rect).astype(np.int0)
            self.rightend=(int((box[2][0]+box[3][0])/2),int((box[2][1]+box[3][1])/2))
        cv2.drawContours(img,[box],0,(155,0,0),3)
        #画Z轴的投影
        print("ZJ point",self.zjointpoint)
        if (self.zwidth==0):
            zwidth=75
        else:
            zwidth=self.zwidth
        w=int(zwidth/2*mt.cos(self.zangle/180*mt.pi))
        h=int(zwidth/2*mt.sin(self.zangle/180*mt.pi))
        self.zcenter=(self.zjointpoint[0]-w,self.zjointpoint[1]-h)
 
        zrect=(self.zcenter, (zwidth,self.zheight), self.zangle)
        print("zrect",zrect)
        zbox = cv2.boxPoints(zrect).astype(np.int0)
        self.zrightend=(int((zbox[0][0]+zbox[1][0])/2),int((zbox[0][1]+zbox[1][1])/2))
        cv2.drawContours(img,[zbox],0,(0,255,0),3)
        
    def setl(self,center,angle):
        self.center=center
        self.angle=angle
    def setangle(self,angle):
        self.angle=angle    
    def setjointpoint(self,jointpoint):
        self.jointpoint=jointpoint
    def setzjointpoint(self,jointpoint):
        self.zjointpoint=jointpoint
    def setzangle(self,angle):
        self.zangle=angle
    def drawrightend(self):
        cv2.circle(img,self.rightend,self.rightendr,(0,0,255),3)
        cv2.circle(img,self.zrightend,self.rightendr,(0,255,0),3)

class Marm():
    def __init__(self,m00,m01,m02,m03,ml01,ml02,ml03):
        self.m00=m00
        self.m01=m01
        self.m02=m02
        self.m03=m03        
        self.ml01=ml01
        self.ml02=ml02
        self.ml03=ml03 
        self.ser=None
        self.img=None
        self.imgcon=None
        self.zl03=75
        #self.x0>=self.zl03
        self.x0=self.zl03
        self.y0=0
        self.xmm=0
        self.ymm=0
        self.zmm=0
        #模拟器初始化
        self.simuinit()
        #串口及机械臂位置初始化
        self.robotinit()
        
    def robotinit(self):
        #打开串口
        ports = list(serial.tools.list_ports.comports())
        print (ports)
        #便利 寻找串口
        for p in ports:
            print (p[1])
            if "serial" in p[1] or "CH340" in p[1]:
                self.ser=serial.Serial(port=p[0])
                if self.ser is not None:
                    print(self.ser)
                    print("serial conected")
                else:
                    print("serial not connected")
                    return
            else:
                print ("No CH340 Device was found connected to the computer")
                return
        if self.ser is None:
            return
        m0=bytes.fromhex('ff 01 00 08 00')# 舵机0速度为 72
        m1=bytes.fromhex('ff 01 01 08 00')# 舵机1速度为 72
        m2=bytes.fromhex('ff 01 02 08 00')# 舵机2速度为 72
        mo0=bytes.fromhex('ff 02 00 dc 05')#设置初始舵机0位置
        mo1=bytes.fromhex('ff 02 01 82 07')#设置初始舵机1的位置
        mo2 = bytes.fromhex('ff 02 02 dc 05')#设置初始舵机2的位置
        mo3 = bytes.fromhex('ff 02 03 dc 05')#设置初始舵机3的位置
        self.ser.write(m0)
        self.ser.write(m1)
        self.ser.write(m2)
        self.ser.write(mo0)
        self.ser.write(mo1)
        self.ser.write(mo2)
        self.ser.write(mo3)
        
    def simuinit(self):
        return
    
    def xztoq0(self,x,z):
        r=mt.sqrt(x*x+z*z+0.01)
        if z >=0:
            q0=180-mt.acos(z/r)/mt.pi*180
        else:
            q0=mt.acos(abs(z)/r)/mt.pi*180
        print("xztoq0,q0 is",q0)
        return q0
    
    def xytoq12(self,x,y):
        L1=self.ml01.width
        L2=self.ml02.width
        L3=mt.sqrt(y*y+x*x)+0.001
        #L3最长不超过两臂之和
        if L3>L1+L2:
            L3=L1+L2-4
        #L3最短不少于两臂之差
        if L3<abs(L1-L2):
            L3=abs(L1-L2)+4
        p=(L1+L2+L3)/2
        S=mt.sqrt(p*(p-L1)*(p-L2)*(p-L3))
        h=S*2/L3
        Q5=mt.asin(h/L2)/mt.pi*180
        Q4=mt.asin(y/L3)/mt.pi*180
        Q7=mt.asin(h/L1)/mt.pi*180
        Q1=180-Q5-Q7
        Q2=Q5+Q4
        #print("Q1,Q2",Q1,Q2)
        return Q1,Q2
    
    #输入参数为屏幕角度，不考虑电机面对面放置,考虑杆的移动相对位置
    def scrdraw(self,angle0,angle1,angle2):
        #电机0在底盘,安装误差10度
        angle0+=10
        self.m00.drawrelative(angle0)
        #杆02与电机0中心在z轴方向相连
        self.ml02.setzjointpoint(self.m00.center)
        self.ml02.setzangle(angle0)
        #电机02
        self.m02.drawrelative(angle2)
         #杆02,与m02的圆心相连
        self.ml02.setangle(angle2)
        self.ml02.draw()
        self.ml02.setjointpoint(self.m02.center)
        #电机01放在杆02的右端
        self.m01.setcenter(self.ml02.rightend)
        self.m01.zcenter=self.ml02.zrightend
        print("m01 zcenter is",self.m01.zcenter)
        self.m01.startangle=180+self.m02.angle
        if angle1<=0:
            angle1=0
        self.m01.drawrelative(angle1)
        #杆01固定在电机01中心，角度与电机01一样
        self.ml01.setjointpoint(self.m01.center)
        self.ml01.setangle(self.m01.angle)
        self.ml01.setzjointpoint(self.m01.zcenter)
        self.ml01.setzangle(angle0)
        self.ml01.draw()
        self.ml01.drawrightend()
        #杆03永远保持垂直，中心点位于杆01右端下方靠左75
        self.ml03.setangle(90)
        L=int(self.ml03.width/2)
        self.ml03.center=np.add(self.ml01.rightend,(-self.zl03,L))
        self.ml03.setzjointpoint(self.ml01.zrightend)
        self.ml03.setzangle(angle0)
        self.ml03.draw()
        self.ml03.drawrightend()
        #显示串口连接状态红未连接绿连接
        if self.ser is not None:
            cv2.circle(img,(50,50),10,(0,255,0),3)
        else:
            cv2.circle(img,(50,50),10,(0,0,255),3)
        
    #输入角度考虑电机是否对面装配，从而是逆时针旋转，然后画在图上   
    def robodraw(self,angle0,angle1,angle2):
        mangle0=angle0
        mangle1=angle1
        mangle2=angle2
        if self.m00.facetome == True:
            print("facetome")
            mangle0=self.m00.maxangle-angle0
        if self.m01.facetome == True:
            print("facetome")
            mangle1=self.m01.maxangle-angle1
        if self.m02.facetome == True:
            print("facetome")
            mangle2=slef.m02.maxangle-angle2
        print("mangel0,mangle1,mangle2",mangle0,mangle1,mangle2)
        self.scrdraw(mangle0,mangle1,mangle2)
    
    #输入PWM值画机械臂。对于非180度舵机实施缩放处理。对零位非0度的情况进行处理
    def drawpwm(self,pwm0,pwm1,pwm2):
        angle0pc=int((pwm0-500)/2000*180)
        angle0pc=180-angle0pc
        angle1pc=int((pwm1-500)/2000*180)
        angle2pc=int((pwm2-500)/2000*180)
        angel0=int(angle0pc*(self.m00.maxangle-self.m00.minangle)/180)+self.m00.minangle
        angel1=int(angle1pc*(self.m01.maxangle-self.m01.minangle)/180)+self.m01.minangle
        angel2=int(angle2pc*(self.m02.maxangle-self.m02.minangle)/180)+self.m02.minangle
        A1=180+50-(angel1+angel2)
        print("angel0,A1,angel2",angel0,A1,angel2)
        self.robodraw(angel0,A1,angel2)
        
    def xytopwm(self,x,y,z):
        rx=mt.sqrt((x+self.x0)*(x+self.x0)+z*z)
        r=rx-self.x0
        q1,q2=self.xytoq12(r,y+self.y0)
        q2=q2-4
        # q1=180+50-(q2+pwmq1)
        #50，20为舵机安装倾斜角度
        pwmq1=180+55-(q1+q2)
        pwmq2=q2
        pwm1=int(pwmq1/180*2000+500)
        pwm2=int(pwmq2/180*2000+500)
        
        q0=self.xztoq0(x+self.x0,z)
        qq0=q0-self.m00.minangle
        pwmq0=qq0/(self.m00.maxangle-self.m00.minangle)*180
        pwm0=int(pwmq0/180*2000+500)
        print("pwm0,pwm1,pwm2",pwm0,pwm1,pwm2)
        return pwm0,pwm1,pwm2
    
    def serwrite(self,pwm0,pwm1,pwm2):
        if self.ser is None:
            return
        print(pwm0,pwm1,pwm2)
        pw0str="%04x"%pwm0
        pw1str="%04x"%pwm1
        pw2str="%04x"%pwm2
        print(pw0str,pw1str,pw2str)
        mo0hexstr="ff 02 00"+" "+pw0str[2:4]+" "+pw0str[0:2]
        mo1hexstr="ff 02 01"+" "+pw1str[2:4]+" "+pw1str[0:2]
        mo2hexstr="ff 02 02"+" "+pw2str[2:4]+" "+pw2str[0:2]
        self.ser.write(bytes.fromhex(mo0hexstr))
        self.ser.write(bytes.fromhex(mo1hexstr))
        self.ser.write(bytes.fromhex(mo2hexstr))
        return
    
    def robotTwin(self,x,y,z):
        print("robotTwin x y z",x,y,z)
        pwm0,pwm1,pwm2=self.xytopwm(x,y,-1*z)
        self.drawpwm(pwm0,pwm1,pwm2)
        self.serwrite(pwm0,pwm1,pwm2)
    
    def robotTwinMM(self,xmm,ymm,zmm):
        self.xmm=xmm
        self.ymm=ymm
        self.zmm=zmm
        x=xmm*200/130
        #补偿越远离缝隙越大
        y=(ymm-0.08*xmm)*200/120
        z=zmm*100/71/(1+1/(xmm/20+3))-1
        self.robotTwin(x,y,z)
        return
    def pumpOn(self,nsec):
        #小于0的值表示一直有气泵
        if self.ser is None:
            return
        mo4 = bytes.fromhex('ff 02 04 c4 09')#设置初始舵机4的位置.180为气泵开
        self.ser.write(mo4)
        time.sleep(nsec)
     
    def pumpOff(self):
        if self.ser is None:
            return
        mo4 = bytes.fromhex('ff 02 04 00 00')#设置初始舵机4的位置.0为气泵关
        self.ser.write(mo4)   
    
    def valve(self,onoff):
        if self.ser is None:
            return
        if onoff == 1:
            mo5=bytes.fromhex('ff 02 05 c4 09')#设置初始舵机5的位置.180为气阀导通，0为气阀阻塞
        else:
            mo5=bytes.fromhex('ff 02 05 00 00')#设置初始舵机5的位置.180为气阀导通，0为气阀阻塞
        self.ser.write(mo5)
    
    def moveto(self,xmm,ymm,zmm):
        #从当前位置抬起
        self.robotTwinMM(self.xmm,self.ymm+50,self.zmm)
        self.show()
        time.sleep(0.5)
        #从空中移动到目的地上方
        self.robotTwinMM(xmm,self.ymm,zmm)
        self.show()
        time.sleep(0.5)
        #到目的地
        self.robotTwinMM(xmm,ymm,zmm)
        self.show()
        time.sleep(0.5)
        
    def show(self):
        global img
        cv2.imshow("marm",img)
        cv2.waitKey(300)
        #恢复背景
        imgcon=getimgcon()
        img=imgcon.copy()
    def home(self):
        self.robotTwin(0,0,0)
        self.pumpOn(1)
        self.valve(0)
        time.sleep(3)
        self.pumpOff()
        self.valve(1)
        self.show()
        cv2.waitKey(5000)
    def destroy(self):
        if self.ser is not None:
            self.ser.close()
        cv2.destroyAllWindows()


def ChessBoard():
    return
        
#配置零件舵机
mM00=mmotor((170+300,240+300),0,0,-70,240)
mM02=mmotor((170+300,240+300),0,0,0,180)
mM01=mmotor((0,0),0,0,0,180)
mM03=mmotor((0,0),0,0,0,180)
#配置零件梁
mMl01=mlever((0,0),200,10,0)
mMl02=mlever((0,0),200,10,0)
mMl03=mlever((0,0),60,10,0)
#组装成机械臂
marm=Marm(mM00,mM01,mM02,mM03,mMl01,mMl02,mMl03)

marm.home()
time.sleep(5)
imgcon=getimgcon()
#cv2.imshow("imgcon",imgcon)
#cv2.waitKey()

marm.robotTwinMM(100,0,-200)
marm.show()
time.sleep(3)
marm.robotTwinMM(100,50,-200)
marm.show()
time.sleep(3)
marm.robotTwinMM(100,50,200)
marm.show()
time.sleep(3)
marm.robotTwinMM(100,0,200)
marm.show()
time.sleep(3)

points=[(50,0,0),(50,0,-150),(150,0,-150),(150,0,0),(150,0,150),(50,0,150)]

while True:
    marm.home()
    marm.show()
    time.sleep(5)
    i=0
    for point in points:
        i+=1
        marm.moveto(point[0],point[1],point[2])
        marm.valve(0)
        marm.pumpOn(3)
        marm.pumpOff()
        marm.moveto(200,20+i*2,200)
        marm.pumpOff()
        marm.valve(1)
        marm.moveto(200,20+i*2,200)
        time.sleep(6)
marm.destroy()
cap.release()

USB-SERIAL CH340 (COM7)
Serial<id=0x1db4bffbc88, open=True>(port='COM7', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)
serial conected
robotTwin x y z 0 0 0
xztoq0,q0 is 90.0
pwm0,pwm1,pwm2 1532 2155 1455
angel0,A1,angel2 81 -3 85
mangel0,mangle1,mangle2 81 -3 85
ZJ point (470, 540)
zrect ((470, 533), (16, 10), 91)
m01 zcenter is (470, 524)
ZJ point (470, 524)
zrect ((470, 531), (-16, 10), 91)
ZJ point (469, 538)
zrect ((469, 501), (75, 10), 91)
1532 2155 1455
05fc 086b 05af


IndexError: list index out of range